# Carga de datos para explosion e ingesta en SQL Server

In [ ]:
import pandas as pd
from openpyxl import load_workbook

# Ruta del archivo
archivo = r"C:\Users\Spider Build\Downloads\Plan Chile\ZMM0164 13.05.2025.xlsx"

# Cargar hoja con openpyxl
wb = load_workbook(archivo, read_only=True)
ws = wb.active

# Buscar la fila donde aparece "Material"
for i, row in enumerate(ws.iter_rows(values_only=True)):
    if row and any(str(cell).strip().lower() == "material" for cell in row):
        header_row = i
        break

# Leer el archivo con pandas desde esa fila
ZMM0164 = pd.read_excel(archivo, engine="openpyxl", skiprows=header_row)

# Elimina columnas sin nombre (NaN o "Unnamed")
ZMM0164 = ZMM0164.loc[:, ~ZMM0164.columns.to_series().astype(str).str.contains("^Unnamed")]

# Elimina la primera fila si todos sus valores son nulos
if ZMM0164.iloc[0].isnull().all():
    ZMM0164 = ZMM0164.iloc[1:]

# Resetear índice después de limpieza (opcional)
ZMM0164.reset_index(drop=True, inplace=True)

# Convertir tipo de datos
ZMM0164["Material"] = ZMM0164["Material"].astype("Int64")
ZMM0164["NºMaterial antiguo"] = ZMM0164["NºMaterial antiguo"].astype("Int64")

# Obtener transacción de kits
ZQUEPP22N = pd.read_excel(r"C:\Users\Spider Build\Downloads\Plan Chile\ZQUEPP22N 13.5.2025.xlsx")



In [ ]:
# Definición de la tabla final a construir, con los productos "explotados"
explosion = pd.DataFrame([])

# Inicialización del dataframe
explosion['cv_id'] = ZMM0164['NºMaterial antiguo']
explosion['cv_prod'] = ZMM0164['NºMaterial antiguo']
explosion['cm_kit'] = ZMM0164['Material']
explosion['cm_prod'] = ZMM0164['Material']
explosion['tipo_prod'] = ZMM0164['TpMt']
explosion['producto'] = ZMM0164['Texto breve de material']
explosion['marca'] = ZMM0164['Marca']
explosion['cantidad'] = ''

df_final = explosion.copy()

iter = 1

while True:
    if iter <= 2:
        # Filtrar registros ZEST
        df_zest = df_final[df_final['tipo_prod'] == 'ZEST'][['cm_kit', 'cantidad']].drop_duplicates()
        print(f'Cantidad de ZEST pendientes: {df_zest['cm_kit'].count()}')

        if df_zest.empty:
            break  # Nada más que desagregar

        # Sí cantidad = '', convertir a 1
        df_zest['cantidad'] = df_zest['cantidad'].replace(['', ' '], None)
        df_zest['cantidad'] = df_zest['cantidad'].fillna(1).astype(int)

        # PASO1: Hacer el left join y seleccionar solo los campos requeridos
        df_joined = df_zest.merge(
            ZQUEPP22N[['Material', 'LisTéc.', 'Componente', 'Qtd.']],
            how='left',
            left_on='cm_kit',
            right_on='Material'
        )[['cm_kit', 'LisTéc.', 'Componente', 'Qtd.', 'cantidad']]    

        # Calcular Qtd. x Cantidad 
        df_joined['cantidad2'] = df_joined['Qtd.']*df_joined['cantidad']
        df_joined = df_joined.drop(columns = ['Qtd.', 'cantidad'])
        df_joined = df_joined.rename(columns = {'cantidad2': 'cantidad'})  

        # Paso 2: Renombrar columnas para nuevo esquema
        df_joined = df_joined.rename(columns={
             'LisTéc.': 'cv_prod',
            'Componente': 'cm_prod',
            'Qtd.': 'cantidad'})

        df_joined["cv_prod"] = df_joined["cv_prod"].astype("Int64")
        df_joined["cm_prod"] = df_joined["cm_prod"].astype("Int64")

        # PASO 3: Hacemos left join con explosion en cm_kit
        df_merged = df_joined.merge(
            explosion,
            how='left',
            on='cm_kit',
            suffixes=('_joined', '_explosion')
        )

        # Reemplazamos los valores de 'cv_prod' y 'cm_prod' de explosion con los de df_joined
        df_merged['cv_prod'] = df_merged['cv_prod_joined']
        df_merged['cm_prod'] = df_merged['cm_prod_joined']
        df_merged['cantidad'] = df_merged['cantidad_joined']

        # Eliminamos las columnas que ya no necesitamos
        df_resultado = df_merged.drop(columns=['cv_prod_joined', 'cm_prod_joined'])

        # Opcionalmente, si quieres reordenar las columnas como en explosion
        df_resultado = df_resultado[explosion.columns]

        # Paso 4: Preparar la tabla de referencia desde explosion
        referencia = explosion[['cm_kit', 'cv_prod', 'tipo_prod', 'producto', 'marca']].drop_duplicates()
        referencia = referencia.rename(columns={'cm_kit': 'cm_prod'})  # porque vamos a unir por cm_prod

        # Paso 5: Eliminar los campos que vamos a reemplazar, si ya existen
        df_base = df_resultado.drop(columns=['cv_prod', 'tipo_prod', 'producto', 'marca'], errors='ignore')

        # Paso 6: Merge con la referencia para traer los valores actualizados
        df_actualizado = df_base.merge(
            referencia,
            how='left',
            on='cm_prod'
        )

        df_final = pd.concat([df_final[df_final['tipo_prod'] != 'ZEST' ], df_actualizado], ignore_index=True).drop_duplicates()

        df_final['cantidad'] = (
            df_final['cantidad']
            .replace(r'^\s*$', None, regex=True)  # Reemplaza cadenas vacías o con espacios por None
            .fillna(1)                            # Rellena los None con 1
            .astype(int)                          # Convierte a entero
        )

        iter = iter + 1

    else:
        df_zest = df_final[df_final['tipo_prod'] == 'ZEST'][['cv_id', 'cm_prod', 'cantidad']]            
        print(f'Cantidad de ZEST pendientes: {df_zest['cm_prod'].count()}')

        if df_zest.empty:
            break  # Nada más que desagregar

        # Sí cantidad = '', convertir a 1
        df_zest['cantidad'] = df_zest['cantidad'].replace(['', ' '], None)
        df_zest['cantidad'] = df_zest['cantidad'].fillna(1).astype(int)

        # PASO1: Hacer el left join y seleccionar solo los campos requeridos
        df_joined = df_zest.merge(
            ZQUEPP22N[['Material', 'LisTéc.', 'Componente', 'Qtd.']],
            how='left',
            left_on='cm_prod',
            right_on='Material'
        )[['cv_id', 'cm_prod', 'LisTéc.', 'Componente', 'Qtd.', 'cantidad']]    

        # Calcular Qtd. x Cantidad 
        df_joined['cantidad2'] = df_joined['Qtd.']*df_joined['cantidad']
        df_joined = df_joined.drop(columns = ['Qtd.', 'cantidad'])
        df_joined = df_joined.rename(columns = {'cantidad2': 'cantidad'})  
    
        # Paso 2: Renombrar columnas para nuevo esquema
        df_joined = df_joined.rename(columns={
            'cm_prod': 'cm_kit',
            'LisTéc.': 'cv_prod',
            'Componente': 'cm_prod',
            'Qtd.': 'cantidad'})

        df_joined["cv_prod"] = df_joined["cv_prod"].astype("Int64")
        df_joined["cm_prod"] = df_joined["cm_prod"].astype("Int64")

        # PASO 3: Hacemos left join con explosion en cm_kit
        df_merged = df_joined.merge(
            explosion,
            how='left',
            on='cm_kit',
            suffixes=('_joined', '_explosion')
        )

        # Reemplazamos los valores de 'cv_prod' y 'cm_prod' de explosion con los de df_joined
        df_merged['cv_prod'] = df_merged['cv_prod_joined']
        df_merged['cm_prod'] = df_merged['cm_prod_joined']
        df_merged['cv_id'] = df_merged['cv_id_joined']
        df_merged['cantidad'] = df_merged['cantidad_joined']

        # Eliminamos las columnas que ya no necesitamos
        df_resultado = df_merged.drop(columns=['cv_prod_joined', 'cm_prod_joined', 'cv_id_joined'])

        # Opcionalmente, si quieres reordenar las columnas como en explosion
        df_resultado = df_resultado[explosion.columns]

        # Paso 4: Preparar la tabla de referencia desde explosion
        referencia = explosion[['cm_kit', 'cv_prod', 'tipo_prod', 'producto', 'marca']].drop_duplicates()
        referencia = referencia.rename(columns={'cm_kit': 'cm_prod'})  # porque vamos a unir por cm_prod

        # Paso 5: Eliminar los campos que vamos a reemplazar, si ya existen
        df_base = df_resultado.drop(columns=['cv_prod', 'tipo_prod', 'producto', 'marca'], errors='ignore')

        # Paso 6: Merge con la referencia para traer los valores actualizados
        df_actualizado = df_base.merge(
            referencia,
            how='left',
            on='cm_prod'
        )

        df_final = pd.concat([df_final[df_final['tipo_prod'] != 'ZEST' ], df_actualizado], ignore_index=True).drop_duplicates()
        print(f'Cantidad de prod resultantes: {df_final['cm_kit'].count()}')

        df_final['cantidad'] = (
            df_final['cantidad']
            .replace(r'^\s*$', None, regex=True)  # Reemplaza cadenas vacías o con espacios por None
            .fillna(1)                            # Rellena los None con 1
            .astype(int)                          # Convierte a entero
        )
        
df_final


In [ ]:
# Ingesta en SQL Server 2019
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import time
import re

# Obtener credenciales SQL Server
# Path to the file
file_path = r"C:\Users\Spider Build\Downloads\SQL2019.txt"

# Function to extract username and password
def extract_credentials(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    
    username_match = re.search(r'username="([^"]+)"', content)
    password_match = re.search(r'password="([^"]+)"', content)
    
    if username_match and password_match:
        username = username_match.group(1)
        password = password_match.group(1)
        return username , password
    else:
        return "Username or password not found in the file."

# Llamar la función de credenciales
username_sql, password_sql = extract_credentials(file_path)

# Función para insertar en SQL Server
def insertar_df_sql(
    df: pd.DataFrame,
    database_name: str,
    schema_name: str,
    table_name: str,
    server_address: str = "10.156.16.46\\SQL2022",
    driver_name: str = "ODBC Driver 17 for SQL Server",
    username_sql: str = None, # Puedes pasarlo como parámetro o cargarlo de forma segura
    password_sql: str = None, # Puedes pasarlo como parámetro o cargarlo de forma segura
    method: str = None, # Puede haber problemas con multi, utilizar None
    if_exists_option: str = 'append',
    chunk_size: int = 1000 # Un chunksize más grande suele ser más eficiente
) -> None:
    """
    Inserta un DataFrame de pandas en una tabla de SQL Server.

    Args:
        df (pd.DataFrame): El DataFrame a insertar.
        database_name (str): El nombre de la base de datos (ej. "CustomerCare").
        schema_name (str): El nombre del esquema (ej. "customer_care").
        table_name (str): El nombre de la tabla (ej. "reclamos").
        server_address (str): Dirección del servidor SQL Server (por defecto "10.156.16.46\\SQL2022").
        driver_name (str): Nombre del driver ODBC (por defecto "ODBC Driver 17 for SQL Server").
        username_sql (str, optional): Nombre de usuario para la conexión SQL. Si no se provee,
                                      la función buscará en variables de entorno o usará autenticación integrada.
        password_sql (str, optional): Contraseña para la conexión SQL.
        if_exists_option (str): Cómo manejar la tabla si ya existe ('fail', 'replace', 'append').
                                Por defecto 'append'.
        chunk_size (int): Número de filas a escribir por lote. Por defecto 1000.
    """
    # --- Configuración de autenticación (ejemplo) ---
    # Construcción de la URL de conexión
    connection_url = URL.create(
        "mssql+pyodbc",
        username=username_sql, # Podría ser None si usas autenticación integrada o variables de entorno
        password=password_sql, # Podría ser None
        host=server_address,
        database=database_name,
        query={
            "driver": driver_name,
            #"TrustServerCertificate": "yes", # Es común necesitar esto si no tienes certificados configurados
            # "authentication": "ActiveDirectoryIntegrated", # Descomenta si usas esta autenticación
        },
    )

    try:
        # Crear el motor de conexión
        engine = create_engine(connection_url)

        with engine.begin() as connection:
            start_time = time.time()
            df.to_sql(
                name=table_name,
                schema=schema_name,
                con=connection,
                method=method,
                chunksize=chunk_size,
                if_exists=if_exists_option,
                index=False, # Generalmente no queremos el índice de pandas como columna en SQL
            )
            elapsed_time = time.time() - start_time
            print(f"{len(df)} Datos insertados en '{database_name}.{schema_name}.{table_name}' exitosamente en {elapsed_time:.2f} segundos.")

    except Exception as e:
        print(f"Error al insertar datos en la base de datos: {e}")

# Ver posibilidades de mejora del method al usar None
#def custom_insert(table, conn, keys, data_iter):
#    from sqlalchemy.dialects.mssql import insert
#    values_list = [dict(zip(keys, row)) for row in data_iter]
#    stmt = insert(table).values(values_list)
#    conn.execute(stmt)

#df.to_sql(
#    name=table_name,
#    schema=schema_name,
#    con=connection,
#    method=custom_insert,
#    chunksize=chunk_size,
#    if_exists=if_exists_option,
#    index=False
#)



In [ ]:
# Transformación de datos
df_processed = df_final.copy() 

df_processed['cv_id'] = df_processed['cv_id'].astype('Int64')
df_processed['cv_prod'] = df_processed['cv_prod'].astype('Int64')
df_processed['cm_kit'] = df_processed['cm_kit'].astype('Int64')
df_processed['cm_prod'] = df_processed['cm_prod'].astype('Int64')
df_processed['tipo_prod'] = df_processed['tipo_prod'].astype(str)
df_processed['producto'] = df_processed['producto'].astype(str)
df_processed['marca'] = df_processed['marca'].astype(str)
df_processed['cantidad'] = df_processed['cantidad'].astype('Int16')

# Insertar tabla productos de de la explosión de panel digital
insertar_df_sql(
        df=df_processed,
        server_address = "10.156.16.45\SQL2019",
        database_name="OyLCL",
        schema_name="plan_chile",
        table_name="productos", # Usa un nombre de tabla temporal para pruebas
        username_sql=username_sql,
        password_sql=password_sql,
        method = None,
        if_exists_option='replace' # 'replace' para crearla de cero cada vez en pruebas
    )

# PROCESAMIENTO RETAIL

In [ ]:
# Procesar explosión demanda de RETAIL 

# Se crea función que carga un .xlsx a dataframe en Pandas, con try & catch.
def crear_dataframe(ruta_archivo):
    """
    Crea un DataFrame de pandas a partir de un archivo Excel.

    Args:
        ruta_archivo (str): La ruta completa del archivo Excel.

    Returns:
        pd.DataFrame: El DataFrame 'productos' creado a partir del archivo,
                      o None si ocurre un error.
    """
    try:
        productos = pd.read_excel(ruta_archivo)
        print("DataFrame creado exitosamente.")
        return productos
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta especificada: {ruta_archivo}")
        return None
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo Excel: {e}")
        return None
    
path_retail = r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Demanda Retail.xlsx"

exp_retail = crear_dataframe(path_retail)
exp_retail

In [ ]:
# Cruzar dataframe exp_retail con los productos a nivel de Código = cv
import numpy as np

# 1. Aislar número
exp_retail['Código'] = exp_retail['Código'].str.replace('P', '').astype(np.int64)
exp_retail

In [ ]:
# 2. Realizar left join con la tabla productos de plan_chile
# Método para obtener df desde SQL Server
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import time

def sql_to_df(
    database_name: str,
    schema_name: str,
    table_name: str = None,
    query: str = None,
    server_address: str = "10.156.16.45\SQL2019",
    driver_name: str = "ODBC Driver 17 for SQL Server",
    username_sql: str = None,
    password_sql: str = None
) -> pd.DataFrame:
    """
    Lee datos de una tabla de SQL Server o ejecuta una consulta SQL y los carga en un DataFrame de pandas.

    Este método establece una conexión con SQL Server utilizando SQLAlchemy y pyodbc.
    Permite tanto la lectura de una tabla completa especificando 'table_name'
    como la ejecución de una consulta SQL personalizada mediante el parámetro 'query'.
    Maneja la construcción de la URL de conexión y proporciona un control básico de errores.

    Args:
        database_name (str): El nombre de la base de datos (ej. "CustomerCare").
        schema_name (str): El nombre del esquema (ej. "customer_care").
        table_name (str, optional): El nombre de la tabla a leer. Se requiere si 'query' no se proporciona.
                                     Si se usa, la función construirá un SELECT * FROM.
        query (str, optional): La consulta SQL a ejecutar. Se requiere si 'table_name' no se proporciona.
                                Se ejecutará la consulta directamente.
        server_address (str): Dirección del servidor SQL Server (por defecto "10.156.16.46\\SQL2022").
        driver_name (str): Nombre del driver ODBC (por defecto "ODBC Driver 17 for SQL Server").
        username_sql (str, optional): Nombre de usuario para la conexión SQL. Si es None,
                                      se intentará la autenticación integrada o se usarán variables de entorno.
        password_sql (str, optional): Contraseña para la conexión SQL. Si es None,
                                      se intentará la autenticación integrada o se usarán variables de entorno.

    Returns:
        pd.DataFrame: Un DataFrame de pandas con los datos leídos de SQL Server.
                      Retorna un DataFrame vacío si ocurre un error o no se encuentran datos.

    Raises:
        ValueError: Si no se proporciona ni 'table_name' ni 'query'.
    """
    if table_name is None and query is None:
        raise ValueError("Debes proporcionar 'table_name' o 'query'.")

    connection_url = URL.create(
        "mssql+pyodbc",
        username=username_sql,
        password=password_sql,
        host=server_address,
        database=database_name,
        query={
            "driver": driver_name,
            # "TrustServerCertificate": "yes", # Descomentar si es necesario para certificados autofirmados/no verificados
            # "authentication": "ActiveDirectoryIntegrated", # Descomentar si usas esta autenticación
        },
    )

    df = pd.DataFrame()
    try:
        engine = create_engine(connection_url)

        start_time = time.time()
        if query:
            df = pd.read_sql_query(sql=query, con=engine)
            print(f"Consulta SQL ejecutada y cargada exitosamente en {time.time() - start_time:.2f} segundos.")
        elif table_name:
            full_table_name = f'"{schema_name}"."{table_name}"'
            df = pd.read_sql_query(sql=f"SELECT * FROM {full_table_name}", con=engine)
            print(f"Tabla '{database_name}.{schema_name}.{table_name}' leída exitosamente en {time.time() - start_time:.2f} segundos.")

    except Exception as e:
        print(f"Error al leer datos de la base de datos: {e}")

    return df

In [ ]:
productos = sql_to_df(
    database_name = 'OyLCL',
    schema_name = 'plan_chile',
    table_name = 'productos',
    query = 'SELECT * FROM OyLCL.plan_chile.productos',
    server_address = "10.156.16.45\SQL2019",
    username_sql = username_sql,
    password_sql = password_sql
)
productos

In [ ]:
# Transformaciones adicionales
productos['cv_id'] = productos['cv_id'].astype('Int64')
productos['cv_prod'] = productos['cv_prod'].astype('Int64')
productos['cm_kit'] = productos['cm_kit'].astype('Int64')
productos['cm_prod'] = productos['cm_prod'].astype('Int64')


In [ ]:
# Left join entre ambos dataframe 
retail_explotado = pd.merge(exp_retail, productos, left_on='Código', right_on='cv_id', how='left')

# Calculando nueva demanda
retail_explotado['Demanda'] = retail_explotado['Quantidade_Itens'] * retail_explotado['cantidad']
retail_explotado

In [ ]:
retail_explotado_final = retail_explotado.drop_duplicates(subset=['Año', 'Ciclo', 'Subciclo', 'Código', 'cv_prod'], keep='first') # Debe ser a nivel de cv_prod y no de kit
retail_explotado_final

In [ ]:
retail_explotado_final[retail_explotado_final['cv_id'] != retail_explotado_final['cv_prod']]

In [ ]:
ZMM0164

In [ ]:
# Para validar que no existan kits
test_retail = pd.merge(ZMM0164, exp_retail, left_on='NºMaterial antiguo', right_on='Código', how='left')
test_retail[test_retail['Código'].notna() & test_retail['TpMt'] == 'ZEST']

In [ ]:
# 3. Construir df final de retail que incluya campos adicionales al .xlsx original
# Crear la columna booleana para 'Canal'
retail_explotado_final['Canal'] = 'RETAIL'

# Seleccionar las columnas deseadas y renombrar 'cv_id' a 'Código'
# Creamos una lista de las columnas que queremos, usando el nuevo nombre para 'cv_id'
df_demanda_retail = retail_explotado_final[[
    'Año',
    'Ciclo',
    'Subciclo',
    'cv_id', # Seleccionamos 'cv_id' primero para luego renombrarlo
    'cv_prod', # Incorporamos el cv del kit y del producto resultante
    'Área',
    'Canal', # Incluimos la nueva columna booleana
    'Demanda'
]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'}) # Renombramos la columna 'cv_id' a 'Código'

# Estandariar campos de año y ciclo
df_demanda_retail['Año'] = df_demanda_retail['Año'].str.replace('FY', '').astype(int) + 2000
df_demanda_retail['Ciclo'] = df_demanda_retail['Ciclo'].str.replace('C', '').str.lstrip('0').astype(int)

df_demanda_retail


In [ ]:
df_demanda_retail.drop(columns = ['Área'], inplace = True)

df_demanda_retail

# Procesamiento VD

In [ ]:
# 1. Obtener los datos de demanda de VD

path_vd = r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Carga de demanda VD.xlsx"

exp_vd = crear_dataframe(path_vd)

# 2. Aislar el número
exp_vd['Cód - Descripción'] = exp_vd['Cód - Descripción'].str.extract('(\d+)')[0].astype('Int64')

# 3. LEFT JOIN con tabla productos
# Left join entre ambos dataframe 
vd_explotado = pd.merge(exp_vd, productos, left_on='Cód - Descripción', right_on='cv_id', how='left')

# Calculando nueva demanda
vd_explotado['Demanda'] = vd_explotado['Cantidad Itens'] * vd_explotado['cantidad']
vd_explotado

In [ ]:
# 4. Filtrar duplicados a nivel de cv_prod y nos quedamos con la primera ocurrencia
vd_explotado_final = vd_explotado.drop_duplicates(subset=['Año', 'Ciclo', 'Subciclo', 'Cód - Descripción', 'cv_prod'], keep='first')
vd_explotado_final

In [ ]:
# 5. Procesamiento de estandarización
# Construir df final de vd que incluya campos adicionales al .xlsx original
# Crear la columna booleana para 'Canal'
vd_explotado_final['Canal'] = 'VD'

# Seleccionar las columnas deseadas y renombrar 'cv_id' a 'Código'
# Creamos una lista de las columnas que queremos, usando el nuevo nombre para 'cv_id'
df_demanda_vd = vd_explotado_final[[
    'Año',
    'Ciclo',
    'Subciclo',
    'cv_id', # Seleccionamos 'cv_id' primero para luego renombrarlo
    'cv_prod', # Incorporamos el cv del kit y del producto resultante
    'Canal', # Incluimos la nueva columna booleana
    'Demanda'
]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'}) # Renombramos la columnas de cv a sus respectivas descripciones

# Estandariar campos de ciclo y subciclo
df_demanda_vd['Ciclo'] = df_demanda_vd['Ciclo'].str.replace('Ciclo ', '').str.lstrip('0').astype(int)
df_demanda_vd['Subciclo'] = df_demanda_vd['Subciclo'].str.replace('Subciclo ', '')
df_demanda_vd['Demanda'] = df_demanda_vd['Demanda'].astype(int)

df_demanda_vd

In [ ]:
# Procesamiento VD Unificado
path_vd = r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Carga de demanda VD.xlsx"

exp_vd = crear_dataframe(path_vd)

# 2. Aislar el número
exp_vd['Cód - Descripción'] = exp_vd['Cód - Descripción'].str.extract('(\d+)')[0].astype('Int64')

# 3. LEFT JOIN con tabla productos
# Left join entre ambos dataframe 
vd_explotado = pd.merge(exp_vd, productos, left_on='Cód - Descripción', right_on='cv_id', how='left')

# Calculando nueva demanda
vd_explotado['Demanda'] = vd_explotado['Cantidad Itens'] * vd_explotado['cantidad']

# 4. Filtrar duplicados a nivel de cv_prod y nos quedamos con la primera ocurrencia
vd_explotado_final = vd_explotado.drop_duplicates(subset=['Año', 'Ciclo', 'Subciclo', 'Cód - Descripción', 'cv_prod'], keep='first')

# 5. Procesamiento de estandarización
# Construir df final de vd que incluya campos adicionales al .xlsx original
# Crear la columna booleana para 'Canal'
vd_explotado_final['Canal'] = 'VD'

# Seleccionar las columnas deseadas y renombrar 'cv_id' a 'Código'
# Creamos una lista de las columnas que queremos, usando el nuevo nombre para 'cv_id'
df_demanda_vd = vd_explotado_final[[
    'Año',
    'Ciclo',
    'Subciclo',
    'cv_id', # Seleccionamos 'cv_id' primero para luego renombrarlo
    'cv_prod', # Incorporamos el cv del kit y del producto resultante
    'Canal', # Incluimos la nueva columna booleana
    'Demanda'
]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'}) # Renombramos la columnas de cv a sus respectivas descripciones

# Estandariar campos de ciclo y subciclo
df_demanda_vd['Ciclo'] = df_demanda_vd['Ciclo'].str.replace('Ciclo ', '').str.lstrip('0').astype(int)
df_demanda_vd['Subciclo'] = df_demanda_vd['Subciclo'].str.replace('Subciclo ', '')
df_demanda_vd['Demanda'] = df_demanda_vd['Demanda'].astype(int)

df_demanda_vd

# Procesamiento VOL

In [ ]:
# 1. Obtener los datos de demanda de VOL

path_vol = r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Carga de demanda Vol.xlsx"

exp_vol = crear_dataframe(path_vol)

# 2. Aislar el número
exp_vol['Código'] = exp_vol['Código'].str.replace('P', '').astype(np.int64)

# 3. LEFT JOIN con tabla productos
# Left join entre ambos dataframe 
vol_explotado = pd.merge(exp_vol, productos, left_on='Código', right_on='cv_id', how='left')

# Calculando nueva demanda
vol_explotado['Demanda'] = vol_explotado['Quantidade_Itens'] * vol_explotado['cantidad']

# 4. Filtrar duplicados a nivel de cv_prod y nos quedamos con la primera ocurrencia
vol_explotado_final = vol_explotado.drop_duplicates(subset=['Año', 'Ciclo', 'Subciclo', 'Código', 'cv_prod'], keep='first')

# 5. Procesamiento de estandarización
# Construir df final de vd que incluya campos adicionales al .xlsx original
# Crear la columna booleana para 'Canal'
vol_explotado_final['Canal'] = 'VOL'

# Seleccionar las columnas deseadas y renombrar 'cv_id' a 'Código'
# Creamos una lista de las columnas que queremos, usando el nuevo nombre para 'cv_id'
df_demanda_vol = vol_explotado_final[[
    'Año',
    'Ciclo',
    'Subciclo',
    'cv_id', # Seleccionamos 'cv_id' primero para luego renombrarlo
    'cv_prod', # Incorporamos el cv del kit y del producto resultante
    'Canal', # Incluimos la nueva columna booleana
    'Demanda'
]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'}) # Renombramos la columnas de cv a sus respectivas descripciones

# Estandariar campos de ciclo y subciclo
# Estandariar campos de año y ciclo
df_demanda_vol['Año'] = df_demanda_vol['Año'].str.replace('FY', '').astype(int) + 2000
df_demanda_vol['Ciclo'] = df_demanda_vol['Ciclo'].str.replace('C', '').str.lstrip('0').astype(int)

df_demanda_vol

# Procesamieto Unificado

In [ ]:
import pandas as pd
import numpy as np

def procesar_demanda_vol(ruta_archivo_vol, df_productos): # Falta validar que sea el mismo código para retail
    """
    Procesa un archivo de demanda de VOL (Venta por Otros Canales) y lo combina
    con un DataFrame de productos para calcular la demanda final y estandarizar
    los datos.

    Args:
        ruta_archivo_vol (str): La ruta completa al archivo Excel (.xlsx) de demanda de VOL.
        df_productos (pd.DataFrame): Un DataFrame de pandas que contiene la
                                      información de los productos, incluyendo
                                      la columna 'cv_id' para el join.

    Returns:
        pd.DataFrame: Un DataFrame procesado con la demanda estandarizada de VOL,
                      incluyendo las columnas 'Año', 'Ciclo', 'Subciclo',
                      'Código_Kit', 'Código_Producto', 'Canal' y 'Demanda'.
    """

    # 1. Obtener los datos de demanda de VOL
    # Se asume que 'crear_dataframe' es una función existente que carga el Excel.
    # Si no, se puede reemplazar por pd.read_excel(ruta_archivo_vol)
    try:
        exp_vol = pd.read_excel(ruta_archivo_vol)
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta: {ruta_archivo_vol}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return pd.DataFrame()

    # 2. Aislar el número del código de producto
    # Se reemplaza 'P' y se convierte a tipo entero de 64 bits para asegurar compatibilidad.
    exp_vol['Código'] = exp_vol['Código'].str.replace('P', '', regex=False).astype(np.int64)

    # 3. LEFT JOIN con tabla productos
    # Combina los DataFrames 'exp_vol' y 'df_productos' usando 'Código' y 'cv_id'.
    # Si 'df_productos' no tiene 'cv_id', se generaría un error.
    if 'cv_id' not in df_productos.columns:
        print("Error: El DataFrame de productos debe contener la columna 'cv_id'.")
        return pd.DataFrame()
    
    vol_explotado = pd.merge(exp_vol, df_productos, left_on='Código', right_on='cv_id', how='left')

    # Calcula la nueva demanda multiplicando la cantidad de ítems por la cantidad del producto.
    vol_explotado['Demanda'] = vol_explotado['Quantidade_Itens'] * vol_explotado['cantidad']

    # 4. Filtrar duplicados a nivel de 'cv_prod'
    # Elimina filas duplicadas basándose en las columnas especificadas,
    # manteniendo solo la primera ocurrencia.
    vol_explotado_final = vol_explotado.drop_duplicates(
        subset=['Año', 'Ciclo', 'Subciclo', 'Código', 'cv_prod'], keep='first'
    )

    # 5. Procesamiento de estandarización
    # Asigna el valor 'VOL' a la nueva columna 'Canal'.
    vol_explotado_final['Canal'] = 'VOL'

    # Selecciona y renombra las columnas finales para el DataFrame de salida.
    df_demanda_vol = vol_explotado_final[[
        'Año',
        'Ciclo',
        'Subciclo',
        'cv_id',
        'cv_prod',
        'Canal',
        'Demanda'
    ]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'})

    # Estandariza los campos de 'Año' y 'Ciclo'.
    # El año se convierte a un formato de cuatro dígitos (ej., 'FY23' a '2023').
    df_demanda_vol['Año'] = df_demanda_vol['Año'].astype(str).str.replace('FY', '', regex=False).astype(int) + 2000
    # El ciclo se convierte a un entero sin ceros iniciales (ej., 'C01' a '1').
    df_demanda_vol['Ciclo'] = df_demanda_vol['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0').astype(int)

    return df_demanda_vol

def procesar_demanda_retail(ruta_archivo_vol, df_productos): # Falta validar que sea el mismo código para retail
    """
    Procesa un archivo de demanda de RETAIL (Venta por Otros Canales) y lo combina
    con un DataFrame de productos para calcular la demanda final y estandarizar
    los datos.

    Args:
        ruta_archivo_vol (str): La ruta completa al archivo Excel (.xlsx) de demanda de RETAIL.
        df_productos (pd.DataFrame): Un DataFrame de pandas que contiene la
                                      información de los productos, incluyendo
                                      la columna 'cv_id' para el join.

    Returns:
        pd.DataFrame: Un DataFrame procesado con la demanda estandarizada de RETAIL,
                      incluyendo las columnas 'Año', 'Ciclo', 'Subciclo',
                      'Código_Kit', 'Código_Producto', 'Canal' y 'Demanda'.
    """

    # 1. Obtener los datos de demanda de RETAIL
    # Se asume que 'crear_dataframe' es una función existente que carga el Excel.
    # Si no, se puede reemplazar por pd.read_excel(ruta_archivo_vol)
    try:
        exp_vol = pd.read_excel(ruta_archivo_vol)
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta: {ruta_archivo_vol}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return pd.DataFrame()

    # 2. Aislar el número del código de producto
    # Se reemplaza 'P' y se convierte a tipo entero de 64 bits para asegurar compatibilidad.
    exp_vol['Código'] = exp_vol['Código'].str.replace('P', '', regex=False).astype(np.int64)

    # 3. LEFT JOIN con tabla productos
    # Combina los DataFrames 'exp_vol' y 'df_productos' usando 'Código' y 'cv_id'.
    # Si 'df_productos' no tiene 'cv_id', se generaría un error.
    if 'cv_id' not in df_productos.columns:
        print("Error: El DataFrame de productos debe contener la columna 'cv_id'.")
        return pd.DataFrame()
    
    vol_explotado = pd.merge(exp_vol, df_productos, left_on='Código', right_on='cv_id', how='left')

    # Calcula la nueva demanda multiplicando la cantidad de ítems por la cantidad del producto.
    vol_explotado['Demanda'] = vol_explotado['Quantidade_Itens'] * vol_explotado['cantidad']

    # 4. Filtrar duplicados a nivel de 'cv_prod'
    # Elimina filas duplicadas basándose en las columnas especificadas,
    # manteniendo solo la primera ocurrencia.
    vol_explotado_final = vol_explotado.drop_duplicates(
        subset=['Año', 'Ciclo', 'Subciclo', 'Código', 'cv_prod'], keep='first'
    )

    # 5. Procesamiento de estandarización
    # Asigna el valor 'VOL' a la nueva columna 'Canal'.
    vol_explotado_final['Canal'] = 'RETAIL'

    # Selecciona y renombra las columnas finales para el DataFrame de salida.
    df_demanda_vol = vol_explotado_final[[
        'Año',
        'Ciclo',
        'Subciclo',
        'cv_id',
        'cv_prod',
        'Canal',
        'Demanda'
    ]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'})

    # Estandariza los campos de 'Año' y 'Ciclo'.
    # El año se convierte a un formato de cuatro dígitos (ej., 'FY23' a '2023').
    df_demanda_vol['Año'] = df_demanda_vol['Año'].astype(str).str.replace('FY', '', regex=False).astype(int) + 2000
    # El ciclo se convierte a un entero sin ceros iniciales (ej., 'C01' a '1').
    df_demanda_vol['Ciclo'] = df_demanda_vol['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0').astype(int)

    return df_demanda_vol

def procesar_demanda_vd(ruta_archivo_vd, df_productos):
    """
    Procesa un archivo de demanda de VD (Venta Directa) y lo combina
    con un DataFrame de productos para calcular la demanda final y estandarizar
    los datos.

    Args:
        ruta_archivo_vd (str): La ruta completa al archivo Excel (.xlsx) de demanda de VD.
        df_productos (pd.DataFrame): Un DataFrame de pandas que contiene la
                                      información de los productos, incluyendo
                                      la columna 'cv_id' para el join.

    Returns:
        pd.DataFrame: Un DataFrame procesado con la demanda estandarizada de VD,
                      incluyendo las columnas 'Año', 'Ciclo', 'Subciclo',
                      'Código_Kit', 'Código_Producto', 'Canal' y 'Demanda'.
    """

    # 1. Obtener los datos de demanda de VD
    # Se asume que 'crear_dataframe' es una función existente que carga el Excel.
    # Si no, se puede reemplazar por pd.read_excel(ruta_archivo_vd)
    try:
        exp_vd = pd.read_excel(ruta_archivo_vd)
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta: {ruta_archivo_vd}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return pd.DataFrame()

    # 2. Aislar el número del código de producto
    # Se extraen los dígitos del código y se convierten a Int64 para manejar valores nulos.
    exp_vd['Cód - Descripción'] = exp_vd['Cód - Descripción'].astype(str).str.extract(r'(\d+)')[0].astype('Int64')

    # 3. LEFT JOIN con tabla productos
    # Se verifica si 'cv_id' existe en el DataFrame de productos antes de intentar el merge.
    if 'cv_id' not in df_productos.columns:
        print("Error: El DataFrame de productos debe contener la columna 'cv_id'.")
        return pd.DataFrame()

    vd_explotado = pd.merge(exp_vd, df_productos, left_on='Cód - Descripción', right_on='cv_id', how='left')

    # Calculando nueva demanda
    vd_explotado['Demanda'] = vd_explotado['Cantidad Itens'] * vd_explotado['cantidad']

    # 4. Filtrar duplicados a nivel de 'cv_prod'
    # Elimina filas duplicadas basándose en las columnas especificadas,
    # manteniendo solo la primera ocurrencia.
    vd_explotado_final = vd_explotado.drop_duplicates(
        subset=['Año', 'Ciclo', 'Subciclo', 'Cód - Descripción', 'cv_prod'], keep='first'
    )

    # 5. Procesamiento de estandarización
    # Asigna el valor 'VD' a la nueva columna 'Canal'.
    vd_explotado_final['Canal'] = 'VD'

    # Selecciona y renombra las columnas finales para el DataFrame de salida.
    df_demanda_vd = vd_explotado_final[[
        'Año',
        'Ciclo',
        'Subciclo',
        'cv_id',
        'cv_prod',
        'Canal',
        'Demanda'
    ]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'})

    # Estandariza los campos de 'Ciclo' y 'Subciclo'.
    # El ciclo se convierte a un entero sin el prefijo 'Ciclo '.
    df_demanda_vd['Ciclo'] = df_demanda_vd['Ciclo'].astype(str).str.replace('Ciclo ', '', regex=False).str.lstrip('0').astype(int)
    # El subciclo se limpia del prefijo 'Subciclo '.
    df_demanda_vd['Subciclo'] = df_demanda_vd['Subciclo'].astype(str).str.replace('Subciclo ', '', regex=False)
    # Asegura que la demanda sea de tipo entero.
    df_demanda_vd['Demanda'] = df_demanda_vd['Demanda'].astype('Int64')

    return df_demanda_vd

In [ ]:
# Demanda de retail
df_demanda_retail = procesar_demanda_retail(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Demanda Retail.xlsx", productos)
df_demanda_retail

In [ ]:
df_demanda_retail[df_demanda_retail['Ciclo'] == 13]

In [ ]:
# Demanda de vol
df_demanda_vol = procesar_demanda_vol(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Carga de demanda Vol.xlsx", productos)
df_demanda_vol

In [ ]:
# Demanda de vd
df_demanda_vd = procesar_demanda_vd(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Carga de demanda VD.xlsx", productos)
df_demanda_vd

# Procesar distribución curva demanda


## Demanda Retail 

In [ ]:
# ESTE MÉTODO FUNCIONA BIEN PARA RETAIL PERO CON DEBUGGER, AÚN HAY ERRORES DE DECIMALES

def distribuir_demanda(df_demanda_retail, ruta_curva_demanda):
    """
    Distribuye la demanda del dataframe df_demanda_retail en los días del ciclo
    según los porcentajes definidos en la curva de demanda.
    Maneja valores de porcentaje con coma decimal y símbolo '%'.
    Estandariza los campos 'Año' y 'Ciclo' en df_curva para que coincidan con df_demanda_retail.

    Args:
        df_demanda_retail (pd.DataFrame): DataFrame con los campos Año, Ciclo, Subciclo,
                                        Código_Kit, Código_Producto, Canal y Demanda.
        ruta_curva_demanda (str): Ruta al archivo Excel (.xlsx) o CSV que contiene la curva de demanda.

    Returns:
        pd.DataFrame: DataFrame con la demanda distribuida por día.
    """
    try:
        # Cargar la curva de demanda.
        if ruta_curva_demanda.endswith('.xlsx'):
            df_curva = pd.read_excel(ruta_curva_demanda)
            print(f"DEBUG: df_curva cargado desde XLSX. Primeras filas ANTES de limpiar y estandarizar:\n{df_curva.head()}")
        elif ruta_curva_demanda.endswith('.csv'):
            df_curva = pd.read_csv(ruta_curva_demanda, decimal=',')
            print(f"DEBUG: df_curva cargado desde CSV. Primeras filas ANTES de limpiar y estandarizar:\n{df_curva.head()}")
        else:
            raise ValueError("Formato de archivo no soportado. Por favor, use .xlsx o .csv")

        # Identificar las columnas que representan los días en la curva de demanda
        columnas_dias_curva = [col for col in df_curva.columns if 'Día' in col]

        if not columnas_dias_curva:
            raise ValueError("No se encontraron columnas de días en la curva de demanda. "
                             "Asegúrese de que los nombres de las columnas contengan 'Día'.")

        # --- INICIO: Estandarización de 'Año' y 'Ciclo' en df_curva ---
        print("\nDEBUG: Estandarizando columnas 'Año' y 'Ciclo' en df_curva...")
        if 'Año' in df_curva.columns:
            # Asumimos que 'Año' en df_curva puede ser 'FY23' o similar.
            # Convertir a string, quitar 'FY', convertir a int, y sumar 2000.
            # Usamos errors='coerce' para convertir a NaN si hay algún valor que no se ajusta
            # y luego rellenamos NaN con un valor por defecto o se gestiona.
            # Para este caso, si no se puede convertir, podría ser un año completo ya.
            df_curva['Año_str_temp'] = df_curva['Año'].astype(str).str.replace('FY', '', regex=False)
            df_curva['Año_temp'] = pd.to_numeric(df_curva['Año_str_temp'], errors='coerce')
            # Si el año ya es 4 dígitos (ej. 2023), no se le suma 2000.
            df_curva['Año'] = df_curva.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
            df_curva.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
            df_curva['Año'] = df_curva['Año'].astype(int) # Asegurar que sea int

            # Pequeña verificación si el año en df_demanda_retail es solo de 2 dígitos,
            # lo que no es el caso en tu ejemplo, pero para robustez.
            # df_demanda_retail['Año'] = df_demanda_retail['Año'].astype(str).apply(lambda x: int(x) + 2000 if len(x) == 2 else int(x))


        if 'Ciclo' in df_curva.columns:
            # Asumimos que 'Ciclo' en df_curva puede ser 'C01' o similar.
            # Convertir a string, quitar 'C', eliminar ceros iniciales, convertir a int.
            df_curva['Ciclo'] = df_curva['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
            df_curva['Ciclo'] = pd.to_numeric(df_curva['Ciclo'], errors='coerce').fillna(0).astype(int) # Coerce y rellenar NaN por si acaso

        print(f"DEBUG: df_curva después de estandarizar 'Año' y 'Ciclo' (primeras filas):\n{df_curva.head()}")
        print(f"DEBUG: Tipos de datos de 'Año' y 'Ciclo' en df_curva: Año={df_curva['Año'].dtype}, Ciclo={df_curva['Ciclo'].dtype}")
        # --- FIN: Estandarización de 'Año' y 'Ciclo' en df_curva ---


        # --- APLICAR LA LIMPIEZA Y CONVERSIÓN DE PORCENTAJES (esto ya estaba y se mantiene) ---
        print(f"\nDEBUG: Procesando columnas de días: {columnas_dias_curva}")
        for col in columnas_dias_curva:
            original_dtype = df_curva[col].dtype
            df_curva[col] = (
                df_curva[col]
                .astype(str)                                # Asegurarse de que sea string
                .str.replace('%', '', regex=False)          # Eliminar el símbolo '%'
                .str.replace(',', '.', regex=False)          # Reemplazar la coma por el punto decimal
            )
            df_curva[col] = pd.to_numeric(df_curva[col], errors='coerce') # Convertir a numérico, NaNs para errores
            print(f"DEBUG: Columna '{col}' - Tipo original: {original_dtype}, Tipo después de limpieza: {df_curva[col].dtype}")

        print(f"\nDEBUG: df_curva después de limpiar porcentajes (primeras filas):\n{df_curva.head()}")
        print(f"DEBUG: Tipos de datos de df_curva después de limpieza:\n{df_curva[columnas_dias_curva].dtypes}")


        # Preparar el dataframe de resultados
        df_resultado = []

        # Iterar sobre cada fila de df_demanda_retail para distribuir la demanda
        for index, row in df_demanda_retail.iterrows():
            # Asegurarse de que 'Año', 'Ciclo', 'Subciclo' de df_demanda_retail sean int para la comparación
            año = row['Año']
            ciclo = row['Ciclo']
            subciclo = row['Subciclo']
            demanda_total = row['Demanda']

            print(f"\nDEBUG: Procesando fila de demanda - Año: {año}, Ciclo: {ciclo}, Subciclo: {subciclo}, Demanda Total: {demanda_total}")

            # Buscar la curva de demanda correspondiente a Año, Ciclo y Subciclo
            curva_especifica = df_curva[
                (df_curva['Año'] == año) &
                (df_curva['Ciclo'] == ciclo) &
                (df_curva['Subciclo'] == subciclo)
            ]

            if not curva_especifica.empty:
                curva_especifica = curva_especifica.iloc[0] # Tomar la primera coincidencia
                print(f"DEBUG: Curva específica ENCONTRADA para {año}-{ciclo}-{subciclo}. Valores de porcentaje:\n{curva_especifica[columnas_dias_curva].to_string()}")

                max_dia_num = 0
                for col_dia in columnas_dias_curva:
                    try:
                        max_dia_num = max(max_dia_num, int(col_dia.replace('Día ', '')))
                    except ValueError:
                        continue
                demanda_distribuida_por_dia = {f'Demanda_Dia_{i+1}' : 0 for i in range(max_dia_num)}


                for col_dia in columnas_dias_curva:
                    try:
                        dia_num = int(col_dia.replace('Día ', ''))
                        valor_porcentaje_curva = curva_especifica[col_dia]
                        porcentaje_dia = valor_porcentaje_curva if pd.notna(valor_porcentaje_curva) else 0 
                        demanda_distribuida = demanda_total * porcentaje_dia

                        print(f"DEBUG: Día: {col_dia}, Valor Curva: {valor_porcentaje_curva}, Porcentaje (decimal): {porcentaje_dia}, Demanda Distribuida: {demanda_distribuida}")

                        demanda_distribuida_por_dia[f'Demanda_Dia_{dia_num}'] = demanda_distribuida
                    except ValueError:
                        continue
                    except KeyError:
                        print(f"ADVERTENCIA: La columna '{col_dia}' no se encontró en la curva específica. Se asignará 0.")
                        # Asegurar que se asigna 0 si el día no existe en la curva (aunque el patrón de columnas_dias_curva debería evitar esto)
                        if f'Demanda_Dia_{dia_num}' not in demanda_distribuida_por_dia:
                             demanda_distribuida_por_dia[f'Demanda_Dia_{dia_num}'] = 0


                nueva_fila = row.to_dict()
                nueva_fila.update(demanda_distribuida_por_dia)
                df_resultado.append(nueva_fila)
            else:
                print(f"DEBUG: NO se encontró curva específica para Año: {año}, Ciclo: {ciclo}, Subciclo: {subciclo}. Asignando 0 a todos los días.")
                nueva_fila = row.to_dict()
                max_dia_num = 0
                for col_dia in columnas_dias_curva:
                    try:
                        max_dia_num = max(max_dia_num, int(col_dia.replace('Día ', '')))
                    except ValueError:
                        continue
                for i in range(max_dia_num):
                     nueva_fila[f'Demanda_Dia_{i+1}'] = 0
                df_resultado.append(nueva_fila)

        return pd.DataFrame(df_resultado)

    except FileNotFoundError:
        print(f"Error: El archivo no fue encontrado en la ruta '{ruta_curva_demanda}'.")
        return pd.DataFrame()
    except KeyError as e:
        print(f"Error: Falta una columna esperada en uno de los DataFrames. Detalle: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return pd.DataFrame()



In [ ]:
# Versión 2 final
import pandas as pd

def distribuir_demanda(df_demanda_retail, ruta_curva_demanda): # Sirve para vol y retail
    """
    Distribuye la demanda del dataframe df_demanda_retail en los días del ciclo
    según los porcentajes definidos en la curva de demanda.
    Maneja valores de porcentaje con coma decimal y símbolo '%'.
    Estandariza los campos 'Año' y 'Ciclo' en df_curva para que coincidan con df_demanda_retail.
    Mantiene los nombres originales de las columnas de días (ej. 'Día 1', 'Día 2').

    Args:
        df_demanda_retail (pd.DataFrame): DataFrame con los campos Año, Ciclo, Subciclo,
                                        Código_Kit, Código_Producto, Canal y Demanda.
        ruta_curva_demanda (str): Ruta al archivo Excel (.xlsx) o CSV que contiene la curva de demanda.

    Returns:
        pd.DataFrame: DataFrame con los datos de df_demanda_retail y las columnas 'Día N'
                      que contienen la demanda distribuida.
    """
    try:
        # Cargar la curva de demanda.
        if ruta_curva_demanda.endswith('.xlsx'):
            df_curva = pd.read_excel(ruta_curva_demanda)
        elif ruta_curva_demanda.endswith('.csv'):
            df_curva = pd.read_csv(ruta_curva_demanda, decimal=',')
        else:
            raise ValueError("Formato de archivo no soportado. Por favor, use .xlsx o .csv")

        # Identificar las columnas que representan los días en la curva de demanda
        columnas_dias_curva = [col for col in df_curva.columns if 'Día' in col]

        if not columnas_dias_curva:
            raise ValueError("No se encontraron columnas de días en la curva de demanda. "
                             "Asegúrese de que los nombres de las columnas contengan 'Día'.")

        # --- Estandarización de 'Año' y 'Ciclo' en df_curva ---
        if 'Año' in df_curva.columns:
            df_curva['Año_str_temp'] = df_curva['Año'].astype(str).str.replace('FY', '', regex=False)
            df_curva['Año_temp'] = pd.to_numeric(df_curva['Año_str_temp'], errors='coerce')
            df_curva['Año'] = df_curva.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
            df_curva.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
            df_curva['Año'] = df_curva['Año'].astype(int)

        if 'Ciclo' in df_curva.columns:
            df_curva['Ciclo'] = df_curva['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
            df_curva['Ciclo'] = pd.to_numeric(df_curva['Ciclo'], errors='coerce').fillna(0).astype(int)
        # --- FIN Estandarización ---

        # --- APLICAR LA LIMPIEZA Y CONVERSIÓN DE PORCENTAJES ---
        for col in columnas_dias_curva:
            df_curva[col] = (
                df_curva[col]
                .astype(str)
                .str.replace('%', '', regex=False)
                .str.replace(',', '.', regex=False)
            )
            df_curva[col] = pd.to_numeric(df_curva[col], errors='coerce')
        # --- FIN Limpieza de Porcentajes ---

        # --- Preparar el DataFrame de resultado ---
        # Obtener el conjunto único de todas las columnas 'Día N' que existen en df_curva
        all_day_columns = sorted([col for col in df_curva.columns if 'Día' in col and col.replace('Día ', '').isdigit()],
                                 key=lambda x: int(x.replace('Día ', '')))
        
        # Crear un df temporal con las columnas de demanda retail + todas las columnas de días inicializadas en 0
        df_resultado_base = df_demanda_retail.copy()
        for col_dia in all_day_columns:
            df_resultado_base[col_dia] = 0.0 # Inicializar con float


        # Iterar sobre cada fila de df_demanda_retail (que ahora es df_resultado_base)
        for index, row in df_resultado_base.iterrows():
            # Asegurarse de que 'Año', 'Ciclo', 'Subciclo' de df_demanda_retail sean int para la comparación
            año = row['Año']
            ciclo = row['Ciclo']
            subciclo = row['Subciclo']
            demanda_total = row['Demanda']

            # Buscar la curva de demanda correspondiente a Año, Ciclo y Subciclo
            curva_especifica = df_curva[
                (df_curva['Año'] == año) &
                (df_curva['Ciclo'] == ciclo) &
                (df_curva['Subciclo'] == subciclo)
            ]

            if not curva_especifica.empty:
                curva_especifica = curva_especifica.iloc[0] # Tomar la primera coincidencia
                
                # Para cada columna de día identificada en la curva de demanda
                for col_dia_name in columnas_dias_curva:
                    try:
                        valor_porcentaje_curva = curva_especifica[col_dia_name]
                        porcentaje_dia = valor_porcentaje_curva if pd.notna(valor_porcentaje_curva) else 0
                        demanda_distribuida = demanda_total * porcentaje_dia

                        # Asignar la demanda distribuida directamente a la columna 'Día N'
                        if col_dia_name in df_resultado_base.columns:
                            df_resultado_base.at[index, col_dia_name] = demanda_distribuida
                        # else: # No es necesario un else aquí, ya que se inicializan todas las columnas
                        #     pass # Si la columna no existe, no se hace nada y se queda con el 0 inicial

                    except KeyError:
                        # Esto ocurriría si col_dia_name existe en columnas_dias_curva pero no en curva_especifica.
                        # Dado cómo se construye columnas_dias_curva, es poco probable para un 'Día N' válido.
                        # Se mantendría el 0 inicializado en df_resultado_base.
                        pass
                    except Exception:
                        # Cualquier otro error durante el cálculo de un día, mantiene el 0 inicializado
                        pass

            # else: # No es necesario un else aquí, las columnas de días ya están en 0 en df_resultado_base
                # Si no se encuentra curva, las columnas de días ya están inicializadas a 0
                # pass

        return df_resultado_base # Retornar el DataFrame base modificado

    except FileNotFoundError:
        print(f"Error: El archivo no fue encontrado en la ruta '{ruta_curva_demanda}'.")
        return pd.DataFrame()
    except KeyError as e:
        print(f"Error: Falta una columna esperada en uno de los DataFrames. Detalle: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return pd.DataFrame()


In [ ]:
pd.set_option('display.max_columns', None)
df = distribuir_demanda(df_demanda_retail, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Retail.xlsx")
df.head(100)

In [ ]:
# Asumiendo que ya tienes el DataFrame llamado df
# Creamos una lista con los nombres de las columnas desde Demanda_Dia_1 hasta Demanda_Dia_50
columnas_a_sumar = [f'Día {i}' for i in range(1, 51)]

# Sumamos esas columnas fila por fila y guardamos el resultado en una nueva columna llamada 'SUMA'
df['SUMA'] = (df[columnas_a_sumar].sum(axis=1)).round(0)
df

In [ ]:
df.to_excel(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\test_curva_retail.xlsx")

In [ ]:
# Leer el Excel normalmente
pd.set_option('display.max_columns', None)
syn = pd.read_excel(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Retail.xlsx")

# Reemplazar '%' y ',' por '.' y convertir a float
columnas_porcentuales = [col for col in syn.columns if 'Día' in col]  # o ajusta según tus nombres

for col in columnas_porcentuales:
    syn[col] = (
        syn[col]
        .astype(str)
        .str.replace('%', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float)
    )
syn

## Obtención de demandas

In [ ]:
# Obtención de cada demanda

demanda_retail = distribuir_demanda(df_demanda_retail, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Retail.xlsx")
demanda_retail


In [ ]:
demanda_vd = distribuir_demanda(df_demanda_vd, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin VD.xlsx")
demanda_vd

In [ ]:
demanda_vol = distribuir_demanda(df_demanda_vol, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Vol.xlsx")
demanda_vol

In [ ]:
# Concatenar los 3 dataframe resultantes


# Supongamos que tus tres DataFrames se llaman df1, df2 y df3
df_combinado = pd.concat([demanda_vol, demanda_retail, demanda_vd], ignore_index=True)
df_combinado.to_excel(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Demanda_Explotada.xlsx", index = False)
df_combinado


# Metodo distribución demanda por día del ciclo

In [ ]:
import pandas as pd
from datetime import datetime

def distribuir_demanda(df_demanda_retail, ruta_curva_demanda, ruta_calendario_ciclos):
    """
    Distribuye la demanda del dataframe df_demanda_retail en los días del ciclo
    según los porcentajes definidos en la curva de demanda y ajusta la distribución
    basándose en la fecha actual y el calendario de ciclos.

    Args:
        df_demanda_retail (pd.DataFrame): DataFrame con los campos Año, Ciclo, Subciclo,
                                          Código_Kit, Código_Producto, Canal y Demanda.
        ruta_curva_demanda (str): Ruta al archivo Excel (.xlsx) o CSV que contiene la curva de demanda.
        ruta_calendario_ciclos (str): Ruta al archivo Excel (.xlsx) o CSV que contiene el calendario de ciclos.

    Returns:
        pd.DataFrame: DataFrame con los datos de df_demanda_retail y las columnas
                      'Día Ciclo' y 'Demanda Día Ciclo' que contienen la demanda distribuida.
    """
    try:
        # Cargar la curva de demanda.
        if ruta_curva_demanda.endswith('.xlsx'):
            df_curva = pd.read_excel(ruta_curva_demanda)
        elif ruta_curva_demanda.endswith('.csv'):
            df_curva = pd.read_csv(ruta_curva_demanda, decimal=',')
        else:
            raise ValueError("Formato de archivo de curva de demanda no soportado. Por favor, use .xlsx o .csv")

        # Cargar el calendario de ciclos.
        if ruta_calendario_ciclos.endswith('.xlsx'):
            df_calendario = pd.read_excel(ruta_calendario_ciclos)
        elif ruta_calendario_ciclos.endswith('.csv'):
            df_calendario = pd.read_csv(ruta_calendario_ciclos, decimal=',')
        else:
            raise ValueError("Formato de archivo de calendario no soportado. Por favor, use .xlsx o .csv")

        # Convertir columnas de fecha en df_calendario a datetime
        df_calendario['Fecha Inicio'] = pd.to_datetime(df_calendario['Fecha Inicio'])
        df_calendario['Fecha Fin'] = pd.to_datetime(df_calendario['Fecha Fin'])

        # Identificar las columnas que representan los días en la curva de demanda
        columnas_dias_curva = [col for col in df_curva.columns if 'Día' in col]

        if not columnas_dias_curva:
            raise ValueError("No se encontraron columnas de días en la curva de demanda. "
                             "Asegúrese de que los nombres de las columnas contengan 'Día'.")

        # --- Estandarización de 'Año' y 'Ciclo' en df_curva ---
        if 'Año' in df_curva.columns:
            df_curva['Año_str_temp'] = df_curva['Año'].astype(str).str.replace('FY', '', regex=False)
            df_curva['Año_temp'] = pd.to_numeric(df_curva['Año_str_temp'], errors='coerce')
            df_curva['Año'] = df_curva.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
            df_curva.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
            df_curva['Año'] = df_curva['Año'].astype(int)

        if 'Ciclo' in df_curva.columns:
            df_curva['Ciclo'] = df_curva['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
            df_curva['Ciclo'] = pd.to_numeric(df_curva['Ciclo'], errors='coerce').fillna(0).astype(int)
        # --- FIN Estandarización ---

        # --- Estandarización de 'Año' y 'Ciclo' en df_calendario (similar a df_curva si es necesario) ---
        if 'Año' in df_calendario.columns:
            df_calendario['Año_str_temp'] = df_calendario['Año'].astype(str).str.replace('FY', '', regex=False)
            df_calendario['Año_temp'] = pd.to_numeric(df_calendario['Año_str_temp'], errors='coerce')
            df_calendario['Año'] = df_calendario.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
            df_calendario.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
            df_calendario['Año'] = df_calendario['Año'].astype(int)

        if 'Ciclo' in df_calendario.columns:
            df_calendario['Ciclo'] = df_calendario['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
            df_calendario['Ciclo'] = pd.to_numeric(df_calendario['Ciclo'], errors='coerce').fillna(0).astype(int)
        # --- FIN Estandarización de calendario ---


        # --- APLICAR LA LIMPIEZA Y CONVERSIÓN DE PORCENTAJES EN df_curva ---
        for col in columnas_dias_curva:
            df_curva[col] = (
                df_curva[col]
                .astype(str)
                .str.replace('%', '', regex=False)
                .str.replace(',', '.', regex=False)
            )
            df_curva[col] = pd.to_numeric(df_curva[col], errors='coerce')
        # --- FIN Limpieza de Porcentajes ---

        # DataFrame final para acumular los resultados
        df_resultado_final = pd.DataFrame(columns=[
            'Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal',
            'Demanda', 'Día Ciclo', 'Demanda Día Ciclo'
        ])

        fecha_hoy = datetime.now() # Ojo: Usa datetime.now() o una fecha fija para pruebas

        # Iterar sobre cada fila de df_demanda_retail
        for index, row in df_demanda_retail.iterrows():
            año = row['Año']
            ciclo = row['Ciclo']
            subciclo = row['Subciclo']
            demanda_total = row['Demanda']

            # Buscar la curva de demanda correspondiente
            curva_especifica = df_curva[
                (df_curva['Año'] == año) &
                (df_curva['Ciclo'] == ciclo) &
                (df_curva['Subciclo'] == subciclo)
            ]

            # Buscar el calendario específico
            calendario_especifico = df_calendario[
                (df_calendario['Año'] == año) &
                (df_calendario['Ciclo'] == ciclo) &
                (df_calendario['Subciclo'] == subciclo)
            ]

            if not curva_especifica.empty and not calendario_especifico.empty:
                curva_especifica = curva_especifica.iloc[0]
                calendario_especifico = calendario_especifico.iloc[0]

                fecha_inicio_ciclo = calendario_especifico['Fecha Inicio']
                fecha_fin_ciclo = calendario_especifico['Fecha Fin']
                dias_curva = calendario_especifico['Dias curva'] # Asumiendo que 'Dias curva' está en el calendario

                demanda_distribuida_por_dia = {}
                
                # Caso 1: fecha_hoy > fecha_fin_ciclo (Ciclo terminado)
                if fecha_hoy > fecha_fin_ciclo:
                    for i in range(1, int(dias_curva) + 1):
                        col_dia_name = f'Día {i}'
                        porcentaje_dia = curva_especifica.get(col_dia_name, 0) # Usar .get para manejar posibles faltantes
                        demanda_distribuida_por_dia[i] = demanda_total * (porcentaje_dia / 100) # Asumir que porcentajes están en %
                
                # Caso 2: fecha_hoy < fecha_inicio_ciclo (Ciclo futuro)
                elif fecha_hoy < fecha_inicio_ciclo:
                    for i in range(1, int(dias_curva) + 1):
                        col_dia_name = f'Día {i}'
                        porcentaje_dia = curva_especifica.get(col_dia_name, 0)
                        demanda_distribuida_por_dia[i] = demanda_total * (porcentaje_dia / 100)
                
                # Caso 3: fecha_inicio_ciclo <= fecha_hoy <= fecha_fin_ciclo (Ciclo en curso)
                else:
                    dias_transcurridos = (fecha_hoy - fecha_inicio_ciclo).days + 1
                    
                    # Calcular la demanda ya "consumida" hasta hoy
                    porcentaje_acumulado_pasado = 0
                    for i in range(1, dias_transcurridos): # Sumar porcentajes de días anteriores a hoy
                        col_dia_name = f'Día {i}'
                        porcentaje_dia = curva_especifica.get(col_dia_name, 0)
                        porcentaje_acumulado_pasado += porcentaje_dia

                    # La demanda pendiente es el total
                    demanda_pendiente = demanda_total

                    # Calcular la suma de los porcentajes de los días restantes
                    suma_porcentajes_pendientes = 0
                    for i in range(dias_transcurridos, int(dias_curva) + 1):
                        col_dia_name = f'Día {i}'
                        porcentaje_dia = curva_especifica.get(col_dia_name, 0)
                        suma_porcentajes_pendientes += porcentaje_dia

                    if suma_porcentajes_pendientes > 0:
                        # Distribuir la demanda pendiente en los días restantes manteniendo la proporción
                        for i in range(1, int(dias_curva) + 1):
                            col_dia_name = f'Día {i}'
                            if i < dias_transcurridos:
                                # Días pasados tienen 0 demanda, se asume que ya se distribuyó
                                demanda_distribuida_por_dia[i] = 0
                            else:
                                porcentaje_dia_original = curva_especifica.get(col_dia_name, 0)
                                # Redistribuir la demanda pendiente
                                demanda_distribuida_por_dia[i] = demanda_pendiente * (porcentaje_dia_original / suma_porcentajes_pendientes)
                    else:
                        # Si no hay días pendientes con porcentaje, la demanda pendiente se vuelve 0 para esos días
                        for i in range(1, int(dias_curva) + 1):
                            demanda_distribuida_por_dia[i] = 0
            else:
                # Si no se encuentra curva o calendario, la demanda por día es 0
                for i in range(1, int(curva_especifica.get('Dias curva', 0)) + 1 if not curva_especifica.empty else 1):
                    demanda_distribuida_por_dia[i] = 0 # Inicializar a 0 si no hay curva/calendario
                
            # Agregar las filas al DataFrame final
            for dia_ciclo, demanda_dia_ciclo in demanda_distribuida_por_dia.items():
                new_row = row.to_dict()
                new_row['Día Ciclo'] = dia_ciclo
                new_row['Demanda Día Ciclo'] = demanda_dia_ciclo
                df_resultado_final = pd.concat([df_resultado_final, pd.DataFrame([new_row])], ignore_index=True)

        return df_resultado_final.round(2) # Redondear a 2 decimales para claridad

    except FileNotFoundError:
        print(f"Error: Uno de los archivos no fue encontrado.")
        return pd.DataFrame()
    except KeyError as e:
        print(f"Error: Falta una columna esperada en uno de los DataFrames. Detalle: {e}")
        return pd.DataFrame()
    except ValueError as e:
        print(f"Error en los datos o formato: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return pd.DataFrame()



In [ ]:

ruta_curva = r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Retail.xlsx"
ruta_calendario = r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Calendario Apertura y Cierre 2025.xlsx"





In [ ]:
df_curva = crear_dataframe(ruta_curva)
df_curva

In [ ]:
df_calendar = crear_dataframe(ruta_calendario)
df_calendar

In [ ]:
# código por parte  // VALIDADO QUE FUNCIONA BIEN
df_calendario = df_calendar


# Convertir columnas de fecha en df_calendario a datetime
df_calendario['Fecha Inicio'] = pd.to_datetime(df_calendario['Fecha Inicio'])
df_calendario['Fecha Fin'] = pd.to_datetime(df_calendario['Fecha Fin'])

        # Identificar las columnas que representan los días en la curva de demanda
columnas_dias_curva = [col for col in df_curva.columns if 'Día' in col]

if not columnas_dias_curva:
    raise ValueError("No se encontraron columnas de días en la curva de demanda. "
                             "Asegúrese de que los nombres de las columnas contengan 'Día'.")

        # --- Estandarización de 'Año' y 'Ciclo' en df_curva ---
if 'Año' in df_curva.columns:
    df_curva['Año_str_temp'] = df_curva['Año'].astype(str).str.replace('FY', '', regex=False)
    df_curva['Año_temp'] = pd.to_numeric(df_curva['Año_str_temp'], errors='coerce')
    df_curva['Año'] = df_curva.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
    df_curva.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
    df_curva['Año'] = df_curva['Año'].astype(int)

if 'Ciclo' in df_curva.columns:
    df_curva['Ciclo'] = df_curva['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
    df_curva['Ciclo'] = pd.to_numeric(df_curva['Ciclo'], errors='coerce').fillna(0).astype(int)
        # --- FIN Estandarización ---

        # --- Estandarización de 'Año' y 'Ciclo' en df_calendario (similar a df_curva si es necesario) ---
if 'Año' in df_calendario.columns:
    df_calendario['Año_str_temp'] = df_calendario['Año'].astype(str).str.replace('FY', '', regex=False)
    df_calendario['Año_temp'] = pd.to_numeric(df_calendario['Año_str_temp'], errors='coerce')
    df_calendario['Año'] = df_calendario.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
    df_calendario.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
    df_calendario['Año'] = df_calendario['Año'].astype(int)

if 'Ciclo' in df_calendario.columns:
    df_calendario['Ciclo'] = df_calendario['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
    df_calendario['Ciclo'] = pd.to_numeric(df_calendario['Ciclo'], errors='coerce').fillna(0).astype(int)
        # --- FIN Estandarización de calendario ---

In [ ]:
# DataFrame final para acumular los resultados
df_resultado_final = pd.DataFrame(columns=[
            'Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal',
            'Demanda', 'Día Ciclo', 'Demanda Día Ciclo'
        ])

fecha_hoy = datetime.now() # Ojo: Usa datetime.now() o una fecha fija para pruebas

# Iterar sobre cada fila de df_demanda_retail
for index, row in df_demanda_retail.iterrows():
    año = row['Año']
    ciclo = row['Ciclo']
    subciclo = row['Subciclo']
    demanda_total = row['Demanda']

    # Buscar la curva de demanda correspondiente
    curva_especifica = df_curva[
        (df_curva['Año'] == año) &
        (df_curva['Ciclo'] == ciclo) &
        (df_curva['Subciclo'] == subciclo)
        ]

    # Buscar el calendario específico
    calendario_especifico = df_calendario[
        (df_calendario['Año'] == año) &
        (df_calendario['Ciclo'] == ciclo) &
        (df_calendario['Subciclo'] == subciclo)
        ]
    
    if not curva_especifica.empty and not calendario_especifico.empty:
        curva_especifica = curva_especifica.iloc[0]
        calendario_especifico = calendario_especifico.iloc[0]

        fecha_inicio_ciclo = calendario_especifico['Fecha Inicio']
        fecha_fin_ciclo = calendario_especifico['Fecha Fin']
        dias_curva = calendario_especifico['Dias curva'] # Asumiendo que 'Dias curva' está en el calendario

        demanda_distribuida_por_dia = {}
                
        # Caso 1: fecha_hoy > fecha_fin_ciclo (Ciclo terminado)
        if fecha_hoy > fecha_fin_ciclo:
            for i in range(1, int(dias_curva) + 1):
                col_dia_name = f'Día {i}'
                porcentaje_dia = curva_especifica.get(col_dia_name, 0) # Usar .get para manejar posibles faltantes
                demanda_distribuida_por_dia[i] = demanda_total * (porcentaje_dia / 100) # Asumir que porcentajes están en %
                
        # Caso 2: fecha_hoy < fecha_inicio_ciclo (Ciclo futuro)
        elif fecha_hoy < fecha_inicio_ciclo:
            for i in range(1, int(dias_curva) + 1):
                col_dia_name = f'Día {i}'
                porcentaje_dia = curva_especifica.get(col_dia_name, 0)
                demanda_distribuida_por_dia[i] = demanda_total * (porcentaje_dia / 100)
                
        # Caso 3: fecha_inicio_ciclo <= fecha_hoy <= fecha_fin_ciclo (Ciclo en curso)
        else:
            dias_transcurridos = (fecha_hoy - fecha_inicio_ciclo).days + 1
                    
            # Calcular la demanda ya "consumida" hasta hoy
            porcentaje_acumulado_pasado = 0
            for i in range(1, dias_transcurridos): # Sumar porcentajes de días anteriores a hoy
                col_dia_name = f'Día {i}'
                porcentaje_dia = curva_especifica.get(col_dia_name, 0)
                porcentaje_acumulado_pasado += porcentaje_dia

            # La demanda pendiente es el total
            demanda_pendiente = demanda_total

            # Calcular la suma de los porcentajes de los días restantes
            suma_porcentajes_pendientes = 0
            for i in range(dias_transcurridos, int(dias_curva) + 1):
                col_dia_name = f'Día {i}'
                porcentaje_dia = curva_especifica.get(col_dia_name, 0)
                suma_porcentajes_pendientes += porcentaje_dia

            if suma_porcentajes_pendientes > 0:
                # Distribuir la demanda pendiente en los días restantes manteniendo la proporción
                for i in range(1, int(dias_curva) + 1):
                    col_dia_name = f'Día {i}'
                    if i < dias_transcurridos:
                        # Días pasados tienen 0 demanda, se asume que ya se distribuyó
                        demanda_distribuida_por_dia[i] = 0
                    else:
                        porcentaje_dia_original = curva_especifica.get(col_dia_name, 0)
                        # Redistribuir la demanda pendiente
                        demanda_distribuida_por_dia[i] = demanda_pendiente * (porcentaje_dia_original / suma_porcentajes_pendientes)
            else:
                # Si no hay días pendientes con porcentaje, la demanda pendiente se vuelve 0 para esos días
                for i in range(1, int(dias_curva) + 1):
                    demanda_distribuida_por_dia[i] = 0
    else:
        # Si no se encuentra curva o calendario, la demanda por día es 0
        for i in range(1, int(curva_especifica.get('Dias curva', 0)) + 1 if not curva_especifica.empty else 1):
            demanda_distribuida_por_dia[i] = 0 # Inicializar a 0 si no hay curva/calendario
                
    # Agregar las filas al DataFrame final
    for dia_ciclo, demanda_dia_ciclo in demanda_distribuida_por_dia.items():
        new_row = row.to_dict()
        new_row['Día Ciclo'] = dia_ciclo
        new_row['Demanda Día Ciclo'] = demanda_dia_ciclo
        df_resultado_final = pd.concat([df_resultado_final, pd.DataFrame([new_row])], ignore_index=True)


In [ ]:
calendario_especifico

In [ ]:
df_curva

In [ ]:
df_calendario

In [ ]:
df_demanda_distribuida = distribuir_demanda(df_demanda_retail, ruta_curva, ruta_calendario)
df_demanda_distribuida

# Distribución demanda por dia de ciclo 2

In [ ]:
# Versión 2 final
import pandas as pd

def distribuir_demanda(df_demanda_retail, ruta_curva_demanda): # Sirve para vol y retail
    """
    Distribuye la demanda del dataframe df_demanda_retail en los días del ciclo
    según los porcentajes definidos en la curva de demanda.
    Maneja valores de porcentaje con coma decimal y símbolo '%'.
    Estandariza los campos 'Año' y 'Ciclo' en df_curva para que coincidan con df_demanda_retail.
    Mantiene los nombres originales de las columnas de días (ej. 'Día 1', 'Día 2').

    Args:
        df_demanda_retail (pd.DataFrame): DataFrame con los campos Año, Ciclo, Subciclo,
                                        Código_Kit, Código_Producto, Canal y Demanda.
        ruta_curva_demanda (str): Ruta al archivo Excel (.xlsx) o CSV que contiene la curva de demanda.

    Returns:
        pd.DataFrame: DataFrame con los datos de df_demanda_retail y las columnas 'Día N'
                      que contienen la demanda distribuida.
    """
    try:
        # Cargar la curva de demanda.
        if ruta_curva_demanda.endswith('.xlsx'):
            df_curva = pd.read_excel(ruta_curva_demanda)
        elif ruta_curva_demanda.endswith('.csv'):
            df_curva = pd.read_csv(ruta_curva_demanda, decimal=',')
        else:
            raise ValueError("Formato de archivo no soportado. Por favor, use .xlsx o .csv")

        # Identificar las columnas que representan los días en la curva de demanda
        columnas_dias_curva = [col for col in df_curva.columns if 'Día' in col]

        if not columnas_dias_curva:
            raise ValueError("No se encontraron columnas de días en la curva de demanda. "
                             "Asegúrese de que los nombres de las columnas contengan 'Día'.")

        # --- Estandarización de 'Año' y 'Ciclo' en df_curva ---
        if 'Año' in df_curva.columns:
            df_curva['Año_str_temp'] = df_curva['Año'].astype(str).str.replace('FY', '', regex=False)
            df_curva['Año_temp'] = pd.to_numeric(df_curva['Año_str_temp'], errors='coerce')
            df_curva['Año'] = df_curva.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
            df_curva.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
            df_curva['Año'] = df_curva['Año'].astype(int)

        if 'Ciclo' in df_curva.columns:
            df_curva['Ciclo'] = df_curva['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
            df_curva['Ciclo'] = pd.to_numeric(df_curva['Ciclo'], errors='coerce').fillna(0).astype(int)
        # --- FIN Estandarización ---

        # --- APLICAR LA LIMPIEZA Y CONVERSIÓN DE PORCENTAJES ---
        for col in columnas_dias_curva:
            df_curva[col] = (
                df_curva[col]
                .astype(str)
                .str.replace('%', '', regex=False)
                .str.replace(',', '.', regex=False)
            )
            df_curva[col] = pd.to_numeric(df_curva[col], errors='coerce')
        # --- FIN Limpieza de Porcentajes ---

        # --- Preparar el DataFrame de resultado ---
        # Obtener el conjunto único de todas las columnas 'Día N' que existen en df_curva
        all_day_columns = sorted([col for col in df_curva.columns if 'Día' in col and col.replace('Día ', '').isdigit()],
                                 key=lambda x: int(x.replace('Día ', '')))
        
        # Crear un df temporal con las columnas de demanda retail + todas las columnas de días inicializadas en 0
        df_resultado_base = df_demanda_retail.copy()
        for col_dia in all_day_columns:
            df_resultado_base[col_dia] = 0.0 # Inicializar con float


        # Iterar sobre cada fila de df_demanda_retail (que ahora es df_resultado_base)
        for index, row in df_resultado_base.iterrows():
            # Asegurarse de que 'Año', 'Ciclo', 'Subciclo' de df_demanda_retail sean int para la comparación
            año = row['Año']
            ciclo = row['Ciclo']
            subciclo = row['Subciclo']
            demanda_total = row['Demanda']

            # Buscar la curva de demanda correspondiente a Año, Ciclo y Subciclo
            curva_especifica = df_curva[
                (df_curva['Año'] == año) &
                (df_curva['Ciclo'] == ciclo) &
                (df_curva['Subciclo'] == subciclo)
            ]

            if not curva_especifica.empty:
                curva_especifica = curva_especifica.iloc[0] # Tomar la primera coincidencia
                
                # Para cada columna de día identificada en la curva de demanda
                for col_dia_name in columnas_dias_curva:
                    try:
                        valor_porcentaje_curva = curva_especifica[col_dia_name]
                        porcentaje_dia = valor_porcentaje_curva if pd.notna(valor_porcentaje_curva) else 0
                        demanda_distribuida = porcentaje_dia

                        # Asignar la demanda distribuida directamente a la columna 'Día N'
                        if col_dia_name in df_resultado_base.columns:
                            df_resultado_base.at[index, col_dia_name] = demanda_distribuida
                        # else: # No es necesario un else aquí, ya que se inicializan todas las columnas
                        #     pass # Si la columna no existe, no se hace nada y se queda con el 0 inicial

                    except KeyError:
                        # Esto ocurriría si col_dia_name existe en columnas_dias_curva pero no en curva_especifica.
                        # Dado cómo se construye columnas_dias_curva, es poco probable para un 'Día N' válido.
                        # Se mantendría el 0 inicializado en df_resultado_base.
                        pass
                    except Exception:
                        # Cualquier otro error durante el cálculo de un día, mantiene el 0 inicializado
                        pass

            # else: # No es necesario un else aquí, las columnas de días ya están en 0 en df_resultado_base
                # Si no se encuentra curva, las columnas de días ya están inicializadas a 0
                # pass

        return df_resultado_base # Retornar el DataFrame base modificado

    except FileNotFoundError:
        print(f"Error: El archivo no fue encontrado en la ruta '{ruta_curva_demanda}'.")
        return pd.DataFrame()
    except KeyError as e:
        print(f"Error: Falta una columna esperada en uno de los DataFrames. Detalle: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return pd.DataFrame()


In [ ]:
demanda_retail = distribuir_demanda(df_demanda_retail, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Retail.xlsx")
demanda_retail

In [ ]:
import pandas as pd

def transformar_df_dias(df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma un DataFrame de Pandas, convirtiendo columnas de días
    ('Día 1', 'Día 2', ..., 'Día N') en una sola columna 'Dia_Ciclo'
    y los valores correspondientes en una columna 'Valor_Dia_Ciclo'.

    Args:
        df (pd.DataFrame): El DataFrame original con las columnas de días.

    Returns:
        pd.DataFrame: El DataFrame transformado con una columna 'Dia_Ciclo'
                      y los valores de demanda por día en 'Valor_Dia_Ciclo'.
    """
    # Identifica las columnas que contienen los datos de los días
    columnas_dias = [col for col in df.columns if col.startswith('Día ')]

    # Identifica las columnas que no se van a "derretir" (mantener como identificadores)
    columnas_id = ['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal', 'Demanda']

    # Utiliza pd.melt para "despivotar" el DataFrame
    df_transformado = pd.melt(df,
                              id_vars=columnas_id,
                              value_vars=columnas_dias,
                              var_name='Dia_Ciclo',
                              value_name='Valor_Dia_Ciclo') # ¡Aquí está el cambio!

    # Extraer el número del día de la columna 'Dia_Ciclo' (ej. 'Día 1' -> 1)
    # y convertirlo a tipo entero para facilitar operaciones futuras
    df_transformado['Dia_Ciclo'] = df_transformado['Dia_Ciclo'].str.replace('Día ', '').astype(int)

    # Reordenar las columnas para que 'Dia_Ciclo' y 'Valor_Dia_Ciclo'
    # estén en el orden deseado
    orden_columnas = ['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto',
                      'Canal', 'Demanda', 'Dia_Ciclo', 'Valor_Dia_Ciclo']
    df_transformado = df_transformado[orden_columnas]

    return df_transformado


In [ ]:
demanda_retail_curva = transformar_df_dias(demanda_retail)
demanda_retail_curva

In [ ]:
# Este método funciona para clasificar los días del ciclo calendario

import pandas as pd
from datetime import timedelta

def incorporar_dia_calendario(
    demanda_df: pd.DataFrame,
    ruta_calendario_excel: str
) -> pd.DataFrame:
    """
    Incorpora el Dia_Calendario calculado al DataFrame de demanda_retail_curva,
    leyendo el calendario desde un archivo Excel (.xlsx) y determinando la fecha
    basada en el Dia_Ciclo dentro del rango Fecha Inicio y Fecha Fin.
    Asume que el archivo Excel del calendario tiene una sola hoja.

    Args:
        demanda_df (pd.DataFrame): DataFrame 'demanda_retail_curva' con columnas
                                   Año, Ciclo, Subciclo, Dia_Ciclo.
                                   (Asumiendo 'Año' en formato 'FYxx', 'Ciclo' en 'Cxx', 'Subciclo' en 'A/B/etc.')
        ruta_calendario_excel (str): La ruta completa al archivo Excel del calendario.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con Dia_Calendario, Dia_Inicio y Dia_Fin
                      incorporados, en formato dd-mm-yyyy.
    """
    print("--- DEBUG: Paso 1 - Inicio de la función ---")
    print("demanda_df original head:\n", demanda_df.head())
    print("demanda_df dtypes:\n", demanda_df.dtypes)
    print("\n")

    try:
        calendario_df_raw = pd.read_excel(ruta_calendario_excel)
        print("--- DEBUG: Paso 2 - calendario_df_raw cargado ---")
        print("calendario_df_raw head:\n", calendario_df_raw.head())
        print("calendario_df_raw dtypes:\n", calendario_df_raw.dtypes)
        print("\n")
    except FileNotFoundError:
        print(f"Error: El archivo Excel no se encontró en la ruta: {ruta_calendario_excel}")
        return demanda_df
    except Exception as e:
        print(f"Error al leer el archivo Excel del calendario: {e}")
        return demanda_df

    # --- Preprocesamiento del calendario_df ---
    calendario_df = calendario_df_raw.copy()

    # Convertir 'Año'
    if 'Año' in calendario_df.columns and calendario_df['Año'].dtype == object and \
       calendario_df['Año'].astype(str).str.startswith('FY').any():
        calendario_df['Año'] = calendario_df['Año'].astype(str).str.replace('FY', '20').astype(int)
    else:
        calendario_df['Año'] = calendario_df['Año'].astype(int)

    # Convertir 'Ciclo'
    if 'Ciclo' in calendario_df.columns and calendario_df['Ciclo'].dtype == object and \
       calendario_df['Ciclo'].astype(str).str.startswith('C').any():
        calendario_df['Ciclo'] = calendario_df['Ciclo'].astype(str).str.replace('C', '').astype(int)
    else:
        calendario_df['Ciclo'] = calendario_df['Ciclo'].astype(int)

    # Asegurarse de que 'Subciclo' sea string y limpiar espacios
    if 'Subciclo' in calendario_df.columns:
        calendario_df['Subciclo'] = calendario_df['Subciclo'].astype(str).str.strip()

    # Convertir 'Fecha Inicio' y 'Fecha Fin' a tipo datetime
    calendario_df['Fecha Inicio'] = pd.to_datetime(calendario_df['Fecha Inicio'], errors='coerce')
    calendario_df['Fecha Fin'] = pd.to_datetime(calendario_df['Fecha Fin'], errors='coerce')

    # Calcular la duración del ciclo en días (inclusive)
    calendario_df['Duracion_Real_Dias'] = (calendario_df['Fecha Fin'] - calendario_df['Fecha Inicio']).dt.days + 1

    print("--- DEBUG: Paso 3 - calendario_df preprocesado ---")
    print("calendario_df head:\n", calendario_df.head())
    print("calendario_df dtypes:\n", calendario_df.dtypes)
    # Verificar NaT en fechas
    print("NaT en Fecha Inicio (calendario_df):", calendario_df['Fecha Inicio'].isna().sum())
    print("NaT en Fecha Fin (calendario_df):", calendario_df['Fecha Fin'].isna().sum())
    print("\n")


    calendario_info = calendario_df[['Año', 'Ciclo', 'Subciclo', 'Fecha Inicio', 'Fecha Fin', 'Duracion_Real_Dias']].copy()

    demanda_df_procesado = demanda_df.copy()

    # Asegurar que las columnas de fusión en demanda_df_procesado tengan el mismo tipo
    demanda_df_procesado['Año'] = demanda_df_procesado['Año'].astype(int)
    demanda_df_procesado['Ciclo'] = demanda_df_procesado['Ciclo'].astype(int)
    demanda_df_procesado['Subciclo'] = demanda_df_procesado['Subciclo'].astype(str).str.strip()


    print("--- DEBUG: Paso 4 - demanda_df_procesado antes de la fusión ---")
    print("demanda_df_procesado head:\n", demanda_df_procesado.head())
    print("demanda_df_procesado dtypes:\n", demanda_df_procesado.dtypes)
    print("\n")

    # Fusionar los DataFrames para obtener las fechas de inicio y fin para cada fila de demanda
    demanda_con_fechas_ciclo = pd.merge(
        demanda_df_procesado,
        calendario_info,
        on=['Año', 'Ciclo', 'Subciclo'],
        how='left'
    )

    print("--- DEBUG: Paso 5 - demanda_con_fechas_ciclo después de la fusión ---")
    print("demanda_con_fechas_ciclo head:\n", demanda_con_fechas_ciclo.head(10)) # Muestra más filas
    print("demanda_con_fechas_ciclo dtypes:\n", demanda_con_fechas_ciclo.dtypes)
    # Verificar cuántas filas tienen NaN en Fecha Inicio o Fecha Fin después de la fusión
    print("Filas con NaN en 'Fecha Inicio' o 'Fecha Fin' después de fusión:",
          demanda_con_fechas_ciclo[['Fecha Inicio', 'Fecha Fin', 'Duracion_Real_Dias']].isna().any(axis=1).sum())
    # Muestra las filas que no encontraron match
    print("Filas sin match en calendario:\n", demanda_con_fechas_ciclo[demanda_con_fechas_ciclo['Fecha Inicio'].isna()])
    print("\n")

    # Función para calcular Dia_Calendario, Dia_Inicio y Dia_Fin
    def calcular_fechas_para_fila(row):
        fecha_inicio_ciclo = row['Fecha Inicio']
        fecha_fin_ciclo = row['Fecha Fin']
        dia_ciclo_demanda = row['Dia_Ciclo']
        duracion_ciclo = row['Duracion_Real_Dias']

        dia_calendario = pd.NaT
        dia_inicio_formato = None
        dia_fin_formato = None

        if pd.notna(fecha_inicio_ciclo) and pd.notna(fecha_fin_ciclo) and pd.notna(dia_ciclo_demanda):
            # Asegurarse de que dia_ciclo_demanda sea un entero antes de la operación
            try:
                dia_ciclo_demanda_int = int(dia_ciclo_demanda)
            except ValueError:
                return pd.Series({
                    'Dia_Calendario': pd.NaT,
                    'Dia_Inicio': None,
                    'Dia_Fin': None
                })

            calculated_date = fecha_inicio_ciclo + timedelta(days=dia_ciclo_demanda_int - 1)

            # Validar rango
            if fecha_inicio_ciclo <= calculated_date <= fecha_fin_ciclo and \
               1 <= dia_ciclo_demanda_int <= duracion_ciclo:
                dia_calendario = calculated_date
            # else: dia_calendario ya es pd.NaT

            if pd.notna(fecha_inicio_ciclo):
                dia_inicio_formato = fecha_inicio_ciclo.strftime('%d-%m-%Y')
            if pd.notna(fecha_fin_ciclo):
                dia_fin_formato = fecha_fin_ciclo.strftime('%d-%m-%Y')

        return pd.Series({
            'Dia_Calendario': dia_calendario,
            'Dia_Inicio': dia_inicio_formato,
            'Dia_Fin': dia_fin_formato
        })

    # Aplicar la función a cada fila del DataFrame
    fechas_calculadas = demanda_con_fechas_ciclo.apply(calcular_fechas_para_fila, axis=1)

    print("--- DEBUG: Paso 6 - fechas_calculadas después de apply ---")
    print("fechas_calculadas head:\n", fechas_calculadas.head(10))
    print("NaT en Dia_Calendario (fechas_calculadas):", fechas_calculadas['Dia_Calendario'].isna().sum())
    print("\n")

    # Unir las nuevas columnas al DataFrame original
    demanda_retail_curva_calendario = pd.concat([demanda_con_fechas_ciclo, fechas_calculadas], axis=1)

    # Eliminar las columnas temporales
    demanda_retail_curva_calendario = demanda_retail_curva_calendario.drop(columns=[
        'Fecha Inicio', 'Fecha Fin', 'Duracion_Real_Dias'
    ])

    # Reordenar las columnas
    cols = demanda_retail_curva_calendario.columns.tolist()
    new_date_cols = ['Dia_Calendario', 'Dia_Inicio', 'Dia_Fin']
    existing_new_date_cols = [col for col in new_date_cols if col in cols]
    for col in existing_new_date_cols:
        if col in cols: # Comprobar de nuevo por si se eliminó en la iteración anterior (no debería ocurrir aquí)
            cols.remove(col)

    if 'Dia_Ciclo' in cols:
        idx_dia_ciclo = cols.index('Dia_Ciclo')
        for i, col in enumerate(existing_new_date_cols):
            cols.insert(idx_dia_ciclo + 1 + i, col)
    else:
        cols.extend(existing_new_date_cols)

    demanda_retail_curva_calendario = demanda_retail_curva_calendario[cols]

    # Formatear la columna 'Dia_Calendario'
    demanda_retail_curva_calendario['Dia_Calendario'] = \
        demanda_retail_curva_calendario['Dia_Calendario'].dt.strftime('%d-%m-%Y').replace({pd.NaT: None})

    print("--- DEBUG: Paso 7 - DataFrame final (primeras 10 filas) ---")
    print(demanda_retail_curva_calendario.head(10))
    print("NaT/None en Dia_Calendario (final):", demanda_retail_curva_calendario['Dia_Calendario'].isna().sum())
    print("\n")

    return demanda_retail_curva_calendario



In [ ]:
demanda_retail_curva_calendario = incorporar_dia_calendario(demanda_retail_curva, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Calendario Apertura y Cierre 2025.xlsx")
demanda_retail_curva_calendario

In [ ]:
# Validar calendario
demanda_retail_curva_calendario[demanda_retail_curva_calendario['Ciclo'] == 14][['Ciclo','Dia_Ciclo', 'Dia_Calendario', 'Dia_Inicio', 'Dia_Fin']].drop_duplicates()

In [ ]:
import pandas as pd
from datetime import datetime

def calcular_demanda_por_dia(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula las columnas 'Demanda_Dia_Ciclo' y 'Demanda_Dia_Calendario' en el DataFrame,
    utilizando la fecha actual del sistema.

    Args:
        df (pd.DataFrame): DataFrame de entrada con la información de demanda.

    Returns:
        pd.DataFrame: El DataFrame con las nuevas columnas calculadas.
    """

    # Obtener la fecha de hoy y convertirla a un Timestamp de Pandas
    # --- ¡CORRECCIÓN CLAVE AQUÍ! ---
    fecha_hoy = pd.to_datetime(datetime.now().date())
    print(f"La fecha actual considerada para el cálculo es: {fecha_hoy.strftime('%Y-%m-%d')}")
    # --- FIN DE LA CORRECCIÓN ---


    # Asegúrate de que las columnas de fecha sean tipo datetime y con el formato correcto
    df['Dia_Inicio'] = pd.to_datetime(df['Dia_Inicio'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Fin'] = pd.to_datetime(df['Dia_Fin'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Calendario'] = pd.to_datetime(df['Dia_Calendario'], format='%d-%m-%Y', errors='coerce')

    # 1. Calcular Demanda_Dia_Ciclo
    df['Demanda_Dia_Ciclo'] = df['Demanda'] * df['Valor_Dia_Ciclo']

    # 2. Inicializar Demanda_Dia_Calendario con Demanda_Dia_Ciclo
    df['Demanda_Dia_Calendario'] = df['Demanda_Dia_Ciclo']

    # Función auxiliar para aplicar la lógica de cálculo por grupo
    def _calcular_demanda_grupo(group: pd.DataFrame) -> pd.DataFrame:
        group = group.copy() # Trabajar con una copia para evitar SettingWithCopyWarning

        # Días desde fecha_hoy hasta Dia_Fin: Recalcular
        # Ahora comparamos Timestamp con Timestamp, lo cual es compatible
        mask_dias_futuros = (group['Dia_Calendario'] >= fecha_hoy) & \
                            (group['Dia_Calendario'] <= group['Dia_Fin'].iloc[0])


        if mask_dias_futuros.any():
            # Demanda acumulada hasta el día anterior a fecha_hoy
            mask_dias_pasados = (group['Dia_Calendario'] < fecha_hoy)
            demanda_acumulada_pasada = group.loc[mask_dias_pasados, 'Demanda_Dia_Ciclo'].sum()

            # Demanda total del ciclo (asumimos que es la misma para todo el grupo)
            demanda_total_ciclo = group['Demanda'].iloc[0]

            # Demanda restante a distribuir
            demanda_restante_a_distribuir = demanda_total_ciclo - demanda_acumulada_pasada

            # Suma de los 'Valor_Dia_Ciclo' para los días futuros dentro del grupo
            suma_valor_dia_ciclo_futuro = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'].sum()

            if suma_valor_dia_ciclo_futuro > 0:
                # Recalcular Valor_Dia_Ciclo y luego Demanda_Dia_Calendario para estos días
                recalibrated_valor_dia_ciclo = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'] / suma_valor_dia_ciclo_futuro
                group.loc[mask_dias_futuros, 'Demanda_Dia_Calendario'] = demanda_restante_a_distribuir * recalibrated_valor_dia_ciclo
            else:
                # Si no hay Valor_Dia_Ciclo en el futuro, no hay nada que distribuir en esa lógica.
                # Se mantendrá la Demanda_Dia_Ciclo original para esos días.
                pass
        
        return group

    # Aplicar la función por grupo
    df_resultado = df.groupby(['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal'], group_keys=False).apply(_calcular_demanda_grupo)

    # Asegurarse de que el orden de las columnas se mantenga o se añadan al final
    return df_resultado.reset_index(drop=True)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta # Importar timedelta para sumar días

def calcular_demanda_por_dia(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula las columnas 'Demanda_Dia_Ciclo' y 'Demanda_Dia_Calendario' en el DataFrame,
    utilizando la fecha actual del sistema.

    Args:
        df (pd.DataFrame): DataFrame de entrada con la información de demanda.

    Returns:
        pd.DataFrame: El DataFrame con las nuevas columnas calculadas.
    """

    # Obtener la fecha de hoy y convertirla a un Timestamp de Pandas
    fecha_hoy = pd.to_datetime(datetime.now().date())
    print(f"La fecha actual considerada para el cálculo es: {fecha_hoy.strftime('%Y-%m-%d')}")

    # Asegúrate de que las columnas de fecha sean tipo datetime y con el formato correcto
    df['Dia_Inicio'] = pd.to_datetime(df['Dia_Inicio'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Fin'] = pd.to_datetime(df['Dia_Fin'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Calendario'] = pd.to_datetime(df['Dia_Calendario'], format='%d-%m-%Y', errors='coerce')

    # 1. Calcular Demanda_Dia_Ciclo (Esto se mantiene igual)
    df['Demanda_Dia_Ciclo'] = df['Demanda'] * df['Valor_Dia_Ciclo']

    # 2. Inicializar Demanda_Dia_Calendario con Demanda_Dia_Ciclo
    # Para los días que NO caen en el rango de redistribución, este será el valor final.
    df['Demanda_Dia_Calendario'] = df['Demanda_Dia_Ciclo']

    # Función auxiliar para aplicar la lógica de cálculo por grupo
    def _calcular_demanda_grupo(group: pd.DataFrame) -> pd.DataFrame:
        group = group.copy() # Trabajar con una copia para evitar SettingWithCopyWarning

        # Aseguramos que Dia_Fin para el grupo se tome correctamente
        dia_inicio_grupo = group['Dia_Inicio'].iloc[0]
        dia_fin_grupo = group['Dia_Fin'].iloc[0]
        demanda_total_ciclo = group['Demanda'].iloc[0]

        # Verificar si la fecha de hoy cae dentro del rango [Dia_Inicio, Dia_Fin] del ciclo
        if fecha_hoy >= dia_inicio_grupo and fecha_hoy <= dia_fin_grupo:
            # 1. Calcular la demanda ya "consumida" hasta el día anterior a hoy
            # Los días anteriores a fecha_hoy mantienen su Demanda_Dia_Ciclo original.
            mask_dias_pasados = (group['Dia_Calendario'] < fecha_hoy)
            demanda_acumulada_pasada = group.loc[mask_dias_pasados, 'Demanda_Dia_Ciclo'].sum()

            # Asignar los valores de los días pasados a Demanda_Dia_Calendario
            group.loc[mask_dias_pasados, 'Demanda_Dia_Calendario'] = group.loc[mask_dias_pasados, 'Demanda_Dia_Ciclo']

            # 2. Calcular la demanda restante a distribuir
            demanda_restante_a_distribuir = demanda_total_ciclo - demanda_acumulada_pasada

            # 3. Identificar los días futuros (desde mañana hasta Dia_Fin)
            fecha_manana = fecha_hoy + timedelta(days=1)
            mask_dias_futuros = (group['Dia_Calendario'] >= fecha_manana) & \
                                (group['Dia_Calendario'] <= dia_fin_grupo)

            # Suma de los Valor_Dia_Ciclo para los días futuros originales
            suma_valor_dia_ciclo_futuro_original = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'].sum()

            if suma_valor_dia_ciclo_futuro_original > 0:
                # Recalcular Valor_Dia_Ciclo para los días futuros
                # Los nuevos porcentajes deben sumar 1 (100%) en este rango
                recalibrated_valor_dia_ciclo = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'] / suma_valor_dia_ciclo_futuro_original

                # 4. Distribuir la demanda restante entre los días futuros
                group.loc[mask_dias_futuros, 'Demanda_Dia_Calendario'] = demanda_restante_a_distribuir * recalibrated_valor_dia_ciclo
            else:
                # Si no hay días futuros o sus Valor_Dia_Ciclo suman 0,
                # la demanda restante no se distribuye en esos días.
                # En este caso, la Demanda_Dia_Calendario para esos días futuros seguirá siendo
                # la inicializada (Demanda_Dia_Ciclo), que podría ser cero si Valor_Dia_Ciclo es cero.
                # O si toda la demanda ya se consumió y no hay nada que distribuir, Demanda_Dia_Calendario
                # para los días futuros será 0.
                group.loc[mask_dias_futuros, 'Demanda_Dia_Calendario'] = 0 # No hay Valor_Dia_Ciclo para distribuir.

            # 5. Manejar el 'Dia_Calendario' que corresponde a 'fecha_hoy'
            # Si fecha_hoy coincide con un Dia_Calendario, este día no es "pasado" ni "futuro" para la redistribución.
            # Según tu descripción original, "Los días anteriores a hoy mantendrán la Demanda_Dia_Ciclo."
            # Y la redistribución es "desde el día de hoy hasta el día_fin".
            # Esto implica que el día de hoy puede ser el inicio de la redistribución de lo restante.
            # Sin embargo, tu punto "Los días anteriores a hoy mantendrán la Demanda_Dia_Ciclo" es claro.
            # La nueva especificación dice "demanda_actual considera hasta fecha_hoy. Luego, esta demanda_actual debe distribuirse entre el día de mañana y fecha_fin".
            # Esto significa que el día 'fecha_hoy' no entra en la redistribución de la 'demanda_actual'.
            # Por lo tanto, el día de hoy, si existe en el grupo, debería mantener su Demanda_Dia_Ciclo original.
            mask_hoy = (group['Dia_Calendario'] == fecha_hoy)
            group.loc[mask_hoy, 'Demanda_Dia_Calendario'] = group.loc[mask_hoy, 'Demanda_Dia_Ciclo']


        # Para los casos donde fecha_hoy está fuera del rango [Dia_Inicio, Dia_Fin]
        # (fecha_hoy > dia_fin_grupo o fecha_hoy < dia_inicio_grupo),
        # Demanda_Dia_Calendario ya fue inicializada con Demanda_Dia_Ciclo y se mantiene así.

        return group

    # Aplicar la función por grupo
    df_resultado = df.groupby(['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal'], group_keys=False).apply(_calcular_demanda_grupo)

    # Asegurarse de que el orden de las columnas se mantenga o se añadan al final
    return df_resultado.reset_index(drop=True)

In [ ]:
# Versión de ajustada con condiciones de borde // FUNCIONA BIEN DEJAR
import pandas as pd
from datetime import datetime, timedelta

def calcular_demanda_por_dia(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula las columnas 'Demanda_Dia_Ciclo', 'Demanda_Dia_Calendario' y 'Valor_Dia_Ciclo_Calendario'
    en el DataFrame, utilizando la fecha actual del sistema.

    Args:
        df (pd.DataFrame): DataFrame de entrada con la información de demanda.

    Returns:
        pd.DataFrame: El DataFrame con las nuevas columnas calculadas.
    """

    # Obtener la fecha de hoy y convertirla a un Timestamp de Pandas
    fecha_hoy = pd.to_datetime(datetime.now().date())
    print(f"La fecha actual considerada para el cálculo es: {fecha_hoy.strftime('%Y-%m-%d')}")

    # Asegúrate de que las columnas de fecha sean tipo datetime y con el formato correcto
    df['Dia_Inicio'] = pd.to_datetime(df['Dia_Inicio'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Fin'] = pd.to_datetime(df['Dia_Fin'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Calendario'] = pd.to_datetime(df['Dia_Calendario'], format='%d-%m-%Y', errors='coerce')

    # 1. Calcular Demanda_Dia_Ciclo (Esto se mantiene igual)
    df['Demanda_Dia_Ciclo'] = df['Demanda'] * df['Valor_Dia_Ciclo']

    # 2. Inicializar Demanda_Dia_Calendario con Demanda_Dia_Ciclo y Valor_Dia_Ciclo_Calendario con Valor_Dia_Ciclo
    df['Demanda_Dia_Calendario'] = df['Demanda_Dia_Ciclo']
    df['Valor_Dia_Ciclo_Calendario'] = df['Valor_Dia_Ciclo'] # Inicializar con el valor original

    # Función auxiliar para aplicar la lógica de cálculo por grupo
    def _calcular_demanda_grupo(group: pd.DataFrame) -> pd.DataFrame:
        group = group.copy()

        dia_inicio_grupo = group['Dia_Inicio'].iloc[0]
        dia_fin_grupo = group['Dia_Fin'].iloc[0]
        demanda_total_ciclo = group['Demanda'].iloc[0]

        # Condición principal: Si fecha_hoy está dentro del rango del ciclo [Dia_Inicio, Dia_Fin]
        if fecha_hoy >= dia_inicio_grupo and fecha_hoy <= dia_fin_grupo:
            # Los días hasta e incluyendo fecha_hoy mantienen Demanda_Dia_Ciclo como Demanda_Dia_Calendario
            mask_dias_hasta_hoy = (group['Dia_Calendario'] <= fecha_hoy)
            group.loc[mask_dias_hasta_hoy, 'Demanda_Dia_Calendario'] = group.loc[mask_dias_hasta_hoy, 'Demanda_Dia_Ciclo']
            group.loc[mask_dias_hasta_hoy, 'Valor_Dia_Ciclo_Calendario'] = group.loc[mask_dias_hasta_hoy, 'Valor_Dia_Ciclo']

            # Suma de la demanda hasta fecha_hoy (inclusive)
            demanda_consumida_hasta_hoy = group.loc[mask_dias_hasta_hoy, 'Demanda_Dia_Calendario'].sum()

            # Caso especial: Si fecha_hoy es el último día del ciclo (fecha_fin)
            if fecha_hoy == dia_fin_grupo:
                # No hay demanda pendiente para distribuir, se habrá "cumplido" el 100% de la demanda hasta hoy.
                # Ya los valores de Demanda_Dia_Calendario y Valor_Dia_Ciclo_Calendario fueron establecidos arriba.
                pass
            else:
                # Calcular la demanda pendiente a distribuir (desde mañana hasta fecha_fin)
                demanda_pendiente_a_distribuir = demanda_total_ciclo - demanda_consumida_hasta_hoy
                
                # Asegurarse de que la demanda pendiente no sea negativa
                demanda_pendiente_a_distribuir = max(0, demanda_pendiente_a_distribuir)

                # Días futuros: desde fecha_hoy + 1 hasta fecha_fin
                fecha_manana = fecha_hoy + timedelta(days=1)
                mask_dias_futuros = (group['Dia_Calendario'] >= fecha_manana) & \
                                    (group['Dia_Calendario'] <= dia_fin_grupo)

                # Suma de los Valor_Dia_Ciclo ORIGINALES para los días futuros
                suma_valor_dia_ciclo_futuro_original = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'].sum()

                if suma_valor_dia_ciclo_futuro_original > 0:
                    # Recalcular Valor_Dia_Ciclo_Calendario para los días futuros
                    # Los nuevos porcentajes deben sumar 1 (100%) para la demanda pendiente
                    recalibrated_valor_dia_ciclo = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'] / suma_valor_dia_ciclo_futuro_original
                    group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo_Calendario'] = recalibrated_valor_dia_ciclo

                    # Distribuir la demanda pendiente
                    group.loc[mask_dias_futuros, 'Demanda_Dia_Calendario'] = demanda_pendiente_a_distribuir * recalibrated_valor_dia_ciclo
                else:
                    # Si no hay días futuros con Valor_Dia_Ciclo > 0 para redistribuir,
                    # la demanda pendiente no se distribuye. Estos días quedan en 0 si no tienen valor inicial.
                    group.loc[mask_dias_futuros, 'Demanda_Dia_Calendario'] = 0
                    group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo_Calendario'] = 0
        
        # Casos donde fecha_hoy está fuera del rango [Dia_Inicio, Dia_Fin]
        # (fecha_hoy > dia_fin_grupo o fecha_hoy < dia_inicio_grupo):
        # En estos casos, Demanda_Dia_Calendario y Valor_Dia_Ciclo_Calendario
        # ya fueron inicializados con los valores de Demanda_Dia_Ciclo y Valor_Dia_Ciclo,
        # lo cual es el comportamiento deseado según tus reglas iniciales.

        return group

    # Aplicar la función por grupo
    df_resultado = df.groupby(['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal'], group_keys=False).apply(_calcular_demanda_grupo)

    # Asegurarse de que el orden de las columnas se mantenga o se añadan al final
    return df_resultado.reset_index(drop=True)

In [ ]:
demanda_retail_dia_recalculada = calcular_demanda_por_dia(demanda_retail_curva_calendario)
demanda_retail_dia_recalculada

In [ ]:
demanda_retail_dia_recalculada[
    (demanda_retail_dia_recalculada['Ciclo'] == 10) & 
    (demanda_retail_dia_recalculada['Código_Producto'] == 168788)
]

demanda_retail_dia_recalculada[
    (demanda_retail_dia_recalculada['Ciclo'] == 10) & 
    (demanda_retail_dia_recalculada['Código_Producto'] == 168788)
].to_excel(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\test_curva_retail.xlsx", index = False)

# VERSION COMPLETA FINAL UNIFICADA

## Carga de demandas explotadas por canal

In [ ]:
# Configurar SQL
import re

# Obtener credenciales SQL Server
# Path to the file
file_path = r"C:\Users\Spider Build\Downloads\SQL2019.txt"

# Function to extract username and password
def extract_credentials(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    
    username_match = re.search(r'username="([^"]+)"', content)
    password_match = re.search(r'password="([^"]+)"', content)
    
    if username_match and password_match:
        username = username_match.group(1)
        password = password_match.group(1)
        return username , password
    else:
        return "Username or password not found in the file."

# Llamar la función de credenciales
username_sql, password_sql = extract_credentials(file_path)

In [ ]:
# Obtener dataframe productos desde SQL
# 2. Realizar left join con la tabla productos de plan_chile
# Método para obtener df desde SQL Server
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import time

def sql_to_df(
    database_name: str,
    schema_name: str,
    table_name: str = None,
    query: str = None,
    server_address: str = "10.156.16.45\SQL2019",
    driver_name: str = "ODBC Driver 17 for SQL Server",
    username_sql: str = None,
    password_sql: str = None
) -> pd.DataFrame:
    """
    Lee datos de una tabla de SQL Server o ejecuta una consulta SQL y los carga en un DataFrame de pandas.

    Este método establece una conexión con SQL Server utilizando SQLAlchemy y pyodbc.
    Permite tanto la lectura de una tabla completa especificando 'table_name'
    como la ejecución de una consulta SQL personalizada mediante el parámetro 'query'.
    Maneja la construcción de la URL de conexión y proporciona un control básico de errores.

    Args:
        database_name (str): El nombre de la base de datos (ej. "CustomerCare").
        schema_name (str): El nombre del esquema (ej. "customer_care").
        table_name (str, optional): El nombre de la tabla a leer. Se requiere si 'query' no se proporciona.
                                     Si se usa, la función construirá un SELECT * FROM.
        query (str, optional): La consulta SQL a ejecutar. Se requiere si 'table_name' no se proporciona.
                                Se ejecutará la consulta directamente.
        server_address (str): Dirección del servidor SQL Server (por defecto "10.156.16.46\\SQL2022").
        driver_name (str): Nombre del driver ODBC (por defecto "ODBC Driver 17 for SQL Server").
        username_sql (str, optional): Nombre de usuario para la conexión SQL. Si es None,
                                      se intentará la autenticación integrada o se usarán variables de entorno.
        password_sql (str, optional): Contraseña para la conexión SQL. Si es None,
                                      se intentará la autenticación integrada o se usarán variables de entorno.

    Returns:
        pd.DataFrame: Un DataFrame de pandas con los datos leídos de SQL Server.
                      Retorna un DataFrame vacío si ocurre un error o no se encuentran datos.

    Raises:
        ValueError: Si no se proporciona ni 'table_name' ni 'query'.
    """
    if table_name is None and query is None:
        raise ValueError("Debes proporcionar 'table_name' o 'query'.")

    connection_url = URL.create(
        "mssql+pyodbc",
        username=username_sql,
        password=password_sql,
        host=server_address,
        database=database_name,
        query={
            "driver": driver_name,
            # "TrustServerCertificate": "yes", # Descomentar si es necesario para certificados autofirmados/no verificados
            # "authentication": "ActiveDirectoryIntegrated", # Descomentar si usas esta autenticación
        },
    )

    df = pd.DataFrame()
    try:
        engine = create_engine(connection_url)

        start_time = time.time()
        if query:
            df = pd.read_sql_query(sql=query, con=engine)
            print(f"Consulta SQL ejecutada y cargada exitosamente en {time.time() - start_time:.2f} segundos.")
        elif table_name:
            full_table_name = f'"{schema_name}"."{table_name}"'
            df = pd.read_sql_query(sql=f"SELECT * FROM {full_table_name}", con=engine)
            print(f"Tabla '{database_name}.{schema_name}.{table_name}' leída exitosamente en {time.time() - start_time:.2f} segundos.")

    except Exception as e:
        print(f"Error al leer datos de la base de datos: {e}")

    return df

In [ ]:
# Obtener productos
productos = sql_to_df(
    database_name = 'OyLCL',
    schema_name = 'plan_chile',
    table_name = 'productos',
    query = 'SELECT * FROM OyLCL.plan_chile.productos',
    server_address = "10.156.16.45\SQL2019",
    username_sql = username_sql,
    password_sql = password_sql
)

# Transformaciones adicionales
productos['cv_id'] = productos['cv_id'].astype('Int64')
productos['cv_prod'] = productos['cv_prod'].astype('Int64')
productos['cm_kit'] = productos['cm_kit'].astype('Int64')
productos['cm_prod'] = productos['cm_prod'].astype('Int64')

productos

In [ ]:
import pandas as pd
import numpy as np

def procesar_demanda_vol(ruta_archivo_vol, df_productos): # Falta validar que sea el mismo código para retail
    """
    Procesa un archivo de demanda de VOL (Venta por Otros Canales) y lo combina
    con un DataFrame de productos para calcular la demanda final y estandarizar
    los datos.

    Args:
        ruta_archivo_vol (str): La ruta completa al archivo Excel (.xlsx) de demanda de VOL.
        df_productos (pd.DataFrame): Un DataFrame de pandas que contiene la
                                      información de los productos, incluyendo
                                      la columna 'cv_id' para el join.

    Returns:
        pd.DataFrame: Un DataFrame procesado con la demanda estandarizada de VOL,
                      incluyendo las columnas 'Año', 'Ciclo', 'Subciclo',
                      'Código_Kit', 'Código_Producto', 'Canal' y 'Demanda'.
    """

    # 1. Obtener los datos de demanda de VOL
    # Se asume que 'crear_dataframe' es una función existente que carga el Excel.
    # Si no, se puede reemplazar por pd.read_excel(ruta_archivo_vol)
    try:
        exp_vol = pd.read_excel(ruta_archivo_vol)
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta: {ruta_archivo_vol}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return pd.DataFrame()

    # 2. Aislar el número del código de producto
    # Se reemplaza 'P' y se convierte a tipo entero de 64 bits para asegurar compatibilidad.
    exp_vol['Código'] = exp_vol['Código'].str.replace('P', '', regex=False).astype(np.int64)

    # 3. LEFT JOIN con tabla productos
    # Combina los DataFrames 'exp_vol' y 'df_productos' usando 'Código' y 'cv_id'.
    # Si 'df_productos' no tiene 'cv_id', se generaría un error.
    if 'cv_id' not in df_productos.columns:
        print("Error: El DataFrame de productos debe contener la columna 'cv_id'.")
        return pd.DataFrame()
    
    vol_explotado = pd.merge(exp_vol, df_productos, left_on='Código', right_on='cv_id', how='left')

    # Calcula la nueva demanda multiplicando la cantidad de ítems por la cantidad del producto.
    vol_explotado['Demanda'] = vol_explotado['Quantidade_Itens'] * vol_explotado['cantidad']

    # 4. Filtrar duplicados a nivel de 'cv_prod'
    # Elimina filas duplicadas basándose en las columnas especificadas,
    # manteniendo solo la primera ocurrencia.
    vol_explotado_final = vol_explotado.drop_duplicates(
        subset=['Año', 'Ciclo', 'Subciclo', 'Código', 'cv_prod'], keep='first'
    )

    # 5. Procesamiento de estandarización
    # Asigna el valor 'VOL' a la nueva columna 'Canal'.
    vol_explotado_final['Canal'] = 'VOL'

    # Selecciona y renombra las columnas finales para el DataFrame de salida.
    df_demanda_vol = vol_explotado_final[[
        'Año',
        'Ciclo',
        'Subciclo',
        'cv_id',
        'cv_prod',
        'Canal',
        'Demanda'
    ]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'})

    # Estandariza los campos de 'Año' y 'Ciclo'.
    # El año se convierte a un formato de cuatro dígitos (ej., 'FY23' a '2023').
    df_demanda_vol['Año'] = df_demanda_vol['Año'].astype(str).str.replace('FY', '', regex=False).astype(int) + 2000
    # El ciclo se convierte a un entero sin ceros iniciales (ej., 'C01' a '1').
    df_demanda_vol['Ciclo'] = df_demanda_vol['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0').astype(int)

    return df_demanda_vol

def procesar_demanda_retail(ruta_archivo_vol, df_productos): # Falta validar que sea el mismo código para retail
    """
    Procesa un archivo de demanda de RETAIL (Venta por Otros Canales) y lo combina
    con un DataFrame de productos para calcular la demanda final y estandarizar
    los datos.

    Args:
        ruta_archivo_vol (str): La ruta completa al archivo Excel (.xlsx) de demanda de RETAIL.
        df_productos (pd.DataFrame): Un DataFrame de pandas que contiene la
                                      información de los productos, incluyendo
                                      la columna 'cv_id' para el join.

    Returns:
        pd.DataFrame: Un DataFrame procesado con la demanda estandarizada de RETAIL,
                      incluyendo las columnas 'Año', 'Ciclo', 'Subciclo',
                      'Código_Kit', 'Código_Producto', 'Canal' y 'Demanda'.
    """

    # 1. Obtener los datos de demanda de RETAIL
    # Se asume que 'crear_dataframe' es una función existente que carga el Excel.
    # Si no, se puede reemplazar por pd.read_excel(ruta_archivo_vol)
    try:
        exp_vol = pd.read_excel(ruta_archivo_vol)
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta: {ruta_archivo_vol}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return pd.DataFrame()

    # 2. Aislar el número del código de producto
    # Se reemplaza 'P' y se convierte a tipo entero de 64 bits para asegurar compatibilidad.
    exp_vol['Código'] = exp_vol['Código'].str.replace('P', '', regex=False).astype(np.int64)

    # 3. LEFT JOIN con tabla productos
    # Combina los DataFrames 'exp_vol' y 'df_productos' usando 'Código' y 'cv_id'.
    # Si 'df_productos' no tiene 'cv_id', se generaría un error.
    if 'cv_id' not in df_productos.columns:
        print("Error: El DataFrame de productos debe contener la columna 'cv_id'.")
        return pd.DataFrame()
    
    vol_explotado = pd.merge(exp_vol, df_productos, left_on='Código', right_on='cv_id', how='left')

    # Calcula la nueva demanda multiplicando la cantidad de ítems por la cantidad del producto.
    vol_explotado['Demanda'] = vol_explotado['Quantidade_Itens'] * vol_explotado['cantidad']

    # 4. Filtrar duplicados a nivel de 'cv_prod'
    # Elimina filas duplicadas basándose en las columnas especificadas,
    # manteniendo solo la primera ocurrencia.
    vol_explotado_final = vol_explotado.drop_duplicates(
        subset=['Año', 'Ciclo', 'Subciclo', 'Código', 'cv_prod'], keep='first'
    )

    # 5. Procesamiento de estandarización
    # Asigna el valor 'VOL' a la nueva columna 'Canal'.
    vol_explotado_final['Canal'] = 'RETAIL'

    # Selecciona y renombra las columnas finales para el DataFrame de salida.
    df_demanda_vol = vol_explotado_final[[
        'Año',
        'Ciclo',
        'Subciclo',
        'cv_id',
        'cv_prod',
        'Canal',
        'Demanda'
    ]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'})

    # Estandariza los campos de 'Año' y 'Ciclo'.
    # El año se convierte a un formato de cuatro dígitos (ej., 'FY23' a '2023').
    df_demanda_vol['Año'] = df_demanda_vol['Año'].astype(str).str.replace('FY', '', regex=False).astype(int) + 2000
    # El ciclo se convierte a un entero sin ceros iniciales (ej., 'C01' a '1').
    df_demanda_vol['Ciclo'] = df_demanda_vol['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0').astype(int)

    return df_demanda_vol

def procesar_demanda_vd(ruta_archivo_vd, df_productos):
    """
    Procesa un archivo de demanda de VD (Venta Directa) y lo combina
    con un DataFrame de productos para calcular la demanda final y estandarizar
    los datos.

    Args:
        ruta_archivo_vd (str): La ruta completa al archivo Excel (.xlsx) de demanda de VD.
        df_productos (pd.DataFrame): Un DataFrame de pandas que contiene la
                                      información de los productos, incluyendo
                                      la columna 'cv_id' para el join.

    Returns:
        pd.DataFrame: Un DataFrame procesado con la demanda estandarizada de VD,
                      incluyendo las columnas 'Año', 'Ciclo', 'Subciclo',
                      'Código_Kit', 'Código_Producto', 'Canal' y 'Demanda'.
    """

    # 1. Obtener los datos de demanda de VD
    # Se asume que 'crear_dataframe' es una función existente que carga el Excel.
    # Si no, se puede reemplazar por pd.read_excel(ruta_archivo_vd)
    try:
        exp_vd = pd.read_excel(ruta_archivo_vd)
    except FileNotFoundError:
        print(f"Error: El archivo no se encontró en la ruta: {ruta_archivo_vd}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error al leer el archivo Excel: {e}")
        return pd.DataFrame()

    # 2. Aislar el número del código de producto
    # Se extraen los dígitos del código y se convierten a Int64 para manejar valores nulos.
    exp_vd['Cód - Descripción'] = exp_vd['Cód - Descripción'].astype(str).str.extract(r'(\d+)')[0].astype('Int64')

    # 3. LEFT JOIN con tabla productos
    # Se verifica si 'cv_id' existe en el DataFrame de productos antes de intentar el merge.
    if 'cv_id' not in df_productos.columns:
        print("Error: El DataFrame de productos debe contener la columna 'cv_id'.")
        return pd.DataFrame()

    vd_explotado = pd.merge(exp_vd, df_productos, left_on='Cód - Descripción', right_on='cv_id', how='left')

    # Calculando nueva demanda
    vd_explotado['Demanda'] = vd_explotado['Cantidad Itens'] * vd_explotado['cantidad']

    # 4. Filtrar duplicados a nivel de 'cv_prod'
    # Elimina filas duplicadas basándose en las columnas especificadas,
    # manteniendo solo la primera ocurrencia.
    vd_explotado_final = vd_explotado.drop_duplicates(
        subset=['Año', 'Ciclo', 'Subciclo', 'Cód - Descripción', 'cv_prod'], keep='first'
    )

    # 5. Procesamiento de estandarización
    # Asigna el valor 'VD' a la nueva columna 'Canal'.
    vd_explotado_final['Canal'] = 'VD'

    # Selecciona y renombra las columnas finales para el DataFrame de salida.
    df_demanda_vd = vd_explotado_final[[
        'Año',
        'Ciclo',
        'Subciclo',
        'cv_id',
        'cv_prod',
        'Canal',
        'Demanda'
    ]].rename(columns={'cv_id': 'Código_Kit', 'cv_prod': 'Código_Producto'})

    # Estandariza los campos de 'Ciclo' y 'Subciclo'.
    # El ciclo se convierte a un entero sin el prefijo 'Ciclo '.
    df_demanda_vd['Ciclo'] = df_demanda_vd['Ciclo'].astype(str).str.replace('Ciclo ', '', regex=False).str.lstrip('0').astype(int)
    # El subciclo se limpia del prefijo 'Subciclo '.
    df_demanda_vd['Subciclo'] = df_demanda_vd['Subciclo'].astype(str).str.replace('Subciclo ', '', regex=False)
    # Asegura que la demanda sea de tipo entero.
    df_demanda_vd['Demanda'] = df_demanda_vd['Demanda'].astype('Int64')

    return df_demanda_vd

## Distribución de demandas explotadas en curva estandar

In [ ]:
# Versión 2 final
import pandas as pd

def distribuir_demanda(df_demanda_retail, ruta_curva_demanda): # Sirve para vol y retail
    """
    Distribuye la demanda del dataframe df_demanda_retail en los días del ciclo
    según los porcentajes definidos en la curva de demanda.
    Maneja valores de porcentaje con coma decimal y símbolo '%'.
    Estandariza los campos 'Año' y 'Ciclo' en df_curva para que coincidan con df_demanda_retail.
    Mantiene los nombres originales de las columnas de días (ej. 'Día 1', 'Día 2').

    Args:
        df_demanda_retail (pd.DataFrame): DataFrame con los campos Año, Ciclo, Subciclo,
                                        Código_Kit, Código_Producto, Canal y Demanda.
        ruta_curva_demanda (str): Ruta al archivo Excel (.xlsx) o CSV que contiene la curva de demanda.

    Returns:
        pd.DataFrame: DataFrame con los datos de df_demanda_retail y las columnas 'Día N'
                      que contienen la demanda distribuida.
    """
    try:
        # Cargar la curva de demanda.
        if ruta_curva_demanda.endswith('.xlsx'):
            df_curva = pd.read_excel(ruta_curva_demanda)
        elif ruta_curva_demanda.endswith('.csv'):
            df_curva = pd.read_csv(ruta_curva_demanda, decimal=',')
        else:
            raise ValueError("Formato de archivo no soportado. Por favor, use .xlsx o .csv")

        # Identificar las columnas que representan los días en la curva de demanda
        columnas_dias_curva = [col for col in df_curva.columns if 'Día' in col]

        if not columnas_dias_curva:
            raise ValueError("No se encontraron columnas de días en la curva de demanda. "
                             "Asegúrese de que los nombres de las columnas contengan 'Día'.")

        # --- Estandarización de 'Año' y 'Ciclo' en df_curva ---
        if 'Año' in df_curva.columns:
            df_curva['Año_str_temp'] = df_curva['Año'].astype(str).str.replace('FY', '', regex=False)
            df_curva['Año_temp'] = pd.to_numeric(df_curva['Año_str_temp'], errors='coerce')
            df_curva['Año'] = df_curva.apply(
                lambda row: int(row['Año_temp']) + 2000 if pd.notna(row['Año_temp']) and len(str(int(row['Año_temp']))) <= 2 else int(row['Año']),
                axis=1
            )
            df_curva.drop(columns=['Año_str_temp', 'Año_temp'], inplace=True, errors='ignore')
            df_curva['Año'] = df_curva['Año'].astype(int)

        if 'Ciclo' in df_curva.columns:
            df_curva['Ciclo'] = df_curva['Ciclo'].astype(str).str.replace('C', '', regex=False).str.lstrip('0')
            df_curva['Ciclo'] = pd.to_numeric(df_curva['Ciclo'], errors='coerce').fillna(0).astype(int)
        # --- FIN Estandarización ---

        # --- APLICAR LA LIMPIEZA Y CONVERSIÓN DE PORCENTAJES ---
        for col in columnas_dias_curva:
            df_curva[col] = (
                df_curva[col]
                .astype(str)
                .str.replace('%', '', regex=False)
                .str.replace(',', '.', regex=False)
            )
            df_curva[col] = pd.to_numeric(df_curva[col], errors='coerce')
        # --- FIN Limpieza de Porcentajes ---

        # --- Preparar el DataFrame de resultado ---
        # Obtener el conjunto único de todas las columnas 'Día N' que existen en df_curva
        all_day_columns = sorted([col for col in df_curva.columns if 'Día' in col and col.replace('Día ', '').isdigit()],
                                 key=lambda x: int(x.replace('Día ', '')))
        
        # Crear un df temporal con las columnas de demanda retail + todas las columnas de días inicializadas en 0
        df_resultado_base = df_demanda_retail.copy()
        for col_dia in all_day_columns:
            df_resultado_base[col_dia] = 0.0 # Inicializar con float


        # Iterar sobre cada fila de df_demanda_retail (que ahora es df_resultado_base)
        for index, row in df_resultado_base.iterrows():
            # Asegurarse de que 'Año', 'Ciclo', 'Subciclo' de df_demanda_retail sean int para la comparación
            año = row['Año']
            ciclo = row['Ciclo']
            subciclo = row['Subciclo']
            demanda_total = row['Demanda']

            # Buscar la curva de demanda correspondiente a Año, Ciclo y Subciclo
            curva_especifica = df_curva[
                (df_curva['Año'] == año) &
                (df_curva['Ciclo'] == ciclo) &
                (df_curva['Subciclo'] == subciclo)
            ]

            if not curva_especifica.empty:
                curva_especifica = curva_especifica.iloc[0] # Tomar la primera coincidencia
                
                # Para cada columna de día identificada en la curva de demanda
                for col_dia_name in columnas_dias_curva:
                    try:
                        valor_porcentaje_curva = curva_especifica[col_dia_name]
                        porcentaje_dia = valor_porcentaje_curva if pd.notna(valor_porcentaje_curva) else 0
                        demanda_distribuida = porcentaje_dia

                        # Asignar la demanda distribuida directamente a la columna 'Día N'
                        if col_dia_name in df_resultado_base.columns:
                            df_resultado_base.at[index, col_dia_name] = demanda_distribuida
                        # else: # No es necesario un else aquí, ya que se inicializan todas las columnas
                        #     pass # Si la columna no existe, no se hace nada y se queda con el 0 inicial

                    except KeyError:
                        # Esto ocurriría si col_dia_name existe en columnas_dias_curva pero no en curva_especifica.
                        # Dado cómo se construye columnas_dias_curva, es poco probable para un 'Día N' válido.
                        # Se mantendría el 0 inicializado en df_resultado_base.
                        pass
                    except Exception:
                        # Cualquier otro error durante el cálculo de un día, mantiene el 0 inicializado
                        pass

            # else: # No es necesario un else aquí, las columnas de días ya están en 0 en df_resultado_base
                # Si no se encuentra curva, las columnas de días ya están inicializadas a 0
                # pass

        return df_resultado_base # Retornar el DataFrame base modificado

    except FileNotFoundError:
        print(f"Error: El archivo no fue encontrado en la ruta '{ruta_curva_demanda}'.")
        return pd.DataFrame()
    except KeyError as e:
        print(f"Error: Falta una columna esperada en uno de los DataFrames. Detalle: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return pd.DataFrame()

In [ ]:
import pandas as pd

def transformar_df_dias(df: pd.DataFrame) -> pd.DataFrame:
    """
    Transforma un DataFrame de Pandas, convirtiendo columnas de días
    ('Día 1', 'Día 2', ..., 'Día N') en una sola columna 'Dia_Ciclo'
    y los valores correspondientes en una columna 'Valor_Dia_Ciclo'.

    Args:
        df (pd.DataFrame): El DataFrame original con las columnas de días.

    Returns:
        pd.DataFrame: El DataFrame transformado con una columna 'Dia_Ciclo'
                      y los valores de demanda por día en 'Valor_Dia_Ciclo'.
    """
    # Identifica las columnas que contienen los datos de los días
    columnas_dias = [col for col in df.columns if col.startswith('Día ')]

    # Identifica las columnas que no se van a "derretir" (mantener como identificadores)
    columnas_id = ['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal', 'Demanda']

    # Utiliza pd.melt para "despivotar" el DataFrame
    df_transformado = pd.melt(df,
                              id_vars=columnas_id,
                              value_vars=columnas_dias,
                              var_name='Dia_Ciclo',
                              value_name='Valor_Dia_Ciclo') # ¡Aquí está el cambio!

    # Extraer el número del día de la columna 'Dia_Ciclo' (ej. 'Día 1' -> 1)
    # y convertirlo a tipo entero para facilitar operaciones futuras
    df_transformado['Dia_Ciclo'] = df_transformado['Dia_Ciclo'].str.replace('Día ', '').astype(int)

    # Reordenar las columnas para que 'Dia_Ciclo' y 'Valor_Dia_Ciclo'
    # estén en el orden deseado
    orden_columnas = ['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto',
                      'Canal', 'Demanda', 'Dia_Ciclo', 'Valor_Dia_Ciclo']
    df_transformado = df_transformado[orden_columnas]

    return df_transformado


In [ ]:
# Este método funciona para clasificar los días del ciclo calendario

import pandas as pd
from datetime import timedelta

def incorporar_dia_calendario(
    demanda_df: pd.DataFrame,
    ruta_calendario_excel: str
) -> pd.DataFrame:
    """
    Incorpora el Dia_Calendario calculado al DataFrame de demanda_retail_curva,
    leyendo el calendario desde un archivo Excel (.xlsx) y determinando la fecha
    basada en el Dia_Ciclo dentro del rango Fecha Inicio y Fecha Fin.
    Asume que el archivo Excel del calendario tiene una sola hoja.

    Args:
        demanda_df (pd.DataFrame): DataFrame 'demanda_retail_curva' con columnas
                                   Año, Ciclo, Subciclo, Dia_Ciclo.
                                   (Asumiendo 'Año' en formato 'FYxx', 'Ciclo' en 'Cxx', 'Subciclo' en 'A/B/etc.')
        ruta_calendario_excel (str): La ruta completa al archivo Excel del calendario.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con Dia_Calendario, Dia_Inicio y Dia_Fin
                      incorporados, en formato dd-mm-yyyy.
    """
    print("--- DEBUG: Paso 1 - Inicio de la función ---")
    print("demanda_df original head:\n", demanda_df.head())
    print("demanda_df dtypes:\n", demanda_df.dtypes)
    print("\n")

    try:
        calendario_df_raw = pd.read_excel(ruta_calendario_excel)
        print("--- DEBUG: Paso 2 - calendario_df_raw cargado ---")
        print("calendario_df_raw head:\n", calendario_df_raw.head())
        print("calendario_df_raw dtypes:\n", calendario_df_raw.dtypes)
        print("\n")
    except FileNotFoundError:
        print(f"Error: El archivo Excel no se encontró en la ruta: {ruta_calendario_excel}")
        return demanda_df
    except Exception as e:
        print(f"Error al leer el archivo Excel del calendario: {e}")
        return demanda_df

    # --- Preprocesamiento del calendario_df ---
    calendario_df = calendario_df_raw.copy()

    # Convertir 'Año'
    if 'Año' in calendario_df.columns and calendario_df['Año'].dtype == object and \
       calendario_df['Año'].astype(str).str.startswith('FY').any():
        calendario_df['Año'] = calendario_df['Año'].astype(str).str.replace('FY', '20').astype(int)
    else:
        calendario_df['Año'] = calendario_df['Año'].astype(int)

    # Convertir 'Ciclo'
    if 'Ciclo' in calendario_df.columns and calendario_df['Ciclo'].dtype == object and \
       calendario_df['Ciclo'].astype(str).str.startswith('C').any():
        calendario_df['Ciclo'] = calendario_df['Ciclo'].astype(str).str.replace('C', '').astype(int)
    else:
        calendario_df['Ciclo'] = calendario_df['Ciclo'].astype(int)

    # Asegurarse de que 'Subciclo' sea string y limpiar espacios
    if 'Subciclo' in calendario_df.columns:
        calendario_df['Subciclo'] = calendario_df['Subciclo'].astype(str).str.strip()

    # Convertir 'Fecha Inicio' y 'Fecha Fin' a tipo datetime
    calendario_df['Fecha Inicio'] = pd.to_datetime(calendario_df['Fecha Inicio'], errors='coerce')
    calendario_df['Fecha Fin'] = pd.to_datetime(calendario_df['Fecha Fin'], errors='coerce')

    # Calcular la duración del ciclo en días (inclusive)
    calendario_df['Duracion_Real_Dias'] = (calendario_df['Fecha Fin'] - calendario_df['Fecha Inicio']).dt.days + 1

    print("--- DEBUG: Paso 3 - calendario_df preprocesado ---")
    print("calendario_df head:\n", calendario_df.head())
    print("calendario_df dtypes:\n", calendario_df.dtypes)
    # Verificar NaT en fechas
    print("NaT en Fecha Inicio (calendario_df):", calendario_df['Fecha Inicio'].isna().sum())
    print("NaT en Fecha Fin (calendario_df):", calendario_df['Fecha Fin'].isna().sum())
    print("\n")


    calendario_info = calendario_df[['Año', 'Ciclo', 'Subciclo', 'Fecha Inicio', 'Fecha Fin', 'Duracion_Real_Dias']].copy()

    demanda_df_procesado = demanda_df.copy()

    # Asegurar que las columnas de fusión en demanda_df_procesado tengan el mismo tipo
    demanda_df_procesado['Año'] = demanda_df_procesado['Año'].astype(int)
    demanda_df_procesado['Ciclo'] = demanda_df_procesado['Ciclo'].astype(int)
    demanda_df_procesado['Subciclo'] = demanda_df_procesado['Subciclo'].astype(str).str.strip()


    print("--- DEBUG: Paso 4 - demanda_df_procesado antes de la fusión ---")
    print("demanda_df_procesado head:\n", demanda_df_procesado.head())
    print("demanda_df_procesado dtypes:\n", demanda_df_procesado.dtypes)
    print("\n")

    # Fusionar los DataFrames para obtener las fechas de inicio y fin para cada fila de demanda
    demanda_con_fechas_ciclo = pd.merge(
        demanda_df_procesado,
        calendario_info,
        on=['Año', 'Ciclo', 'Subciclo'],
        how='left'
    )

    print("--- DEBUG: Paso 5 - demanda_con_fechas_ciclo después de la fusión ---")
    print("demanda_con_fechas_ciclo head:\n", demanda_con_fechas_ciclo.head(10)) # Muestra más filas
    print("demanda_con_fechas_ciclo dtypes:\n", demanda_con_fechas_ciclo.dtypes)
    # Verificar cuántas filas tienen NaN en Fecha Inicio o Fecha Fin después de la fusión
    print("Filas con NaN en 'Fecha Inicio' o 'Fecha Fin' después de fusión:",
          demanda_con_fechas_ciclo[['Fecha Inicio', 'Fecha Fin', 'Duracion_Real_Dias']].isna().any(axis=1).sum())
    # Muestra las filas que no encontraron match
    print("Filas sin match en calendario:\n", demanda_con_fechas_ciclo[demanda_con_fechas_ciclo['Fecha Inicio'].isna()])
    print("\n")

    # Función para calcular Dia_Calendario, Dia_Inicio y Dia_Fin
    def calcular_fechas_para_fila(row):
        fecha_inicio_ciclo = row['Fecha Inicio']
        fecha_fin_ciclo = row['Fecha Fin']
        dia_ciclo_demanda = row['Dia_Ciclo']
        duracion_ciclo = row['Duracion_Real_Dias']

        dia_calendario = pd.NaT
        dia_inicio_formato = None
        dia_fin_formato = None

        if pd.notna(fecha_inicio_ciclo) and pd.notna(fecha_fin_ciclo) and pd.notna(dia_ciclo_demanda):
            # Asegurarse de que dia_ciclo_demanda sea un entero antes de la operación
            try:
                dia_ciclo_demanda_int = int(dia_ciclo_demanda)
            except ValueError:
                return pd.Series({
                    'Dia_Calendario': pd.NaT,
                    'Dia_Inicio': None,
                    'Dia_Fin': None
                })

            calculated_date = fecha_inicio_ciclo + timedelta(days=dia_ciclo_demanda_int - 1)

            # Validar rango
            if fecha_inicio_ciclo <= calculated_date <= fecha_fin_ciclo and \
               1 <= dia_ciclo_demanda_int <= duracion_ciclo:
                dia_calendario = calculated_date
            # else: dia_calendario ya es pd.NaT

            if pd.notna(fecha_inicio_ciclo):
                dia_inicio_formato = fecha_inicio_ciclo.strftime('%d-%m-%Y')
            if pd.notna(fecha_fin_ciclo):
                dia_fin_formato = fecha_fin_ciclo.strftime('%d-%m-%Y')

        return pd.Series({
            'Dia_Calendario': dia_calendario,
            'Dia_Inicio': dia_inicio_formato,
            'Dia_Fin': dia_fin_formato
        })

    # Aplicar la función a cada fila del DataFrame
    fechas_calculadas = demanda_con_fechas_ciclo.apply(calcular_fechas_para_fila, axis=1)

    print("--- DEBUG: Paso 6 - fechas_calculadas después de apply ---")
    print("fechas_calculadas head:\n", fechas_calculadas.head(10))
    print("NaT en Dia_Calendario (fechas_calculadas):", fechas_calculadas['Dia_Calendario'].isna().sum())
    print("\n")

    # Unir las nuevas columnas al DataFrame original
    demanda_retail_curva_calendario = pd.concat([demanda_con_fechas_ciclo, fechas_calculadas], axis=1)

    # Eliminar las columnas temporales
    demanda_retail_curva_calendario = demanda_retail_curva_calendario.drop(columns=[
        'Fecha Inicio', 'Fecha Fin', 'Duracion_Real_Dias'
    ])

    # Reordenar las columnas
    cols = demanda_retail_curva_calendario.columns.tolist()
    new_date_cols = ['Dia_Calendario', 'Dia_Inicio', 'Dia_Fin']
    existing_new_date_cols = [col for col in new_date_cols if col in cols]
    for col in existing_new_date_cols:
        if col in cols: # Comprobar de nuevo por si se eliminó en la iteración anterior (no debería ocurrir aquí)
            cols.remove(col)

    if 'Dia_Ciclo' in cols:
        idx_dia_ciclo = cols.index('Dia_Ciclo')
        for i, col in enumerate(existing_new_date_cols):
            cols.insert(idx_dia_ciclo + 1 + i, col)
    else:
        cols.extend(existing_new_date_cols)

    demanda_retail_curva_calendario = demanda_retail_curva_calendario[cols]

    # Formatear la columna 'Dia_Calendario'
    demanda_retail_curva_calendario['Dia_Calendario'] = \
        demanda_retail_curva_calendario['Dia_Calendario'].dt.strftime('%d-%m-%Y').replace({pd.NaT: None})

    print("--- DEBUG: Paso 7 - DataFrame final (primeras 10 filas) ---")
    print(demanda_retail_curva_calendario.head(10))
    print("NaT/None en Dia_Calendario (final):", demanda_retail_curva_calendario['Dia_Calendario'].isna().sum())
    print("\n")

    return demanda_retail_curva_calendario

In [ ]:
# Versión de ajustada con condiciones de borde // FUNCIONA BIEN DEJAR
import pandas as pd
from datetime import datetime, timedelta

def calcular_demanda_por_dia(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula las columnas 'Demanda_Dia_Ciclo', 'Demanda_Dia_Calendario' y 'Valor_Dia_Ciclo_Calendario'
    en el DataFrame, utilizando la fecha actual del sistema.

    Args:
        df (pd.DataFrame): DataFrame de entrada con la información de demanda.

    Returns:
        pd.DataFrame: El DataFrame con las nuevas columnas calculadas.
    """

    # Obtener la fecha de hoy y convertirla a un Timestamp de Pandas
    fecha_hoy = pd.to_datetime(datetime.now().date())
    print(f"La fecha actual considerada para el cálculo es: {fecha_hoy.strftime('%Y-%m-%d')}")

    # Asegúrate de que las columnas de fecha sean tipo datetime y con el formato correcto
    df['Dia_Inicio'] = pd.to_datetime(df['Dia_Inicio'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Fin'] = pd.to_datetime(df['Dia_Fin'], format='%d-%m-%Y', errors='coerce')
    df['Dia_Calendario'] = pd.to_datetime(df['Dia_Calendario'], format='%d-%m-%Y', errors='coerce')

    # 1. Calcular Demanda_Dia_Ciclo (Esto se mantiene igual)
    df['Demanda_Dia_Ciclo'] = df['Demanda'] * df['Valor_Dia_Ciclo']

    # 2. Inicializar Demanda_Dia_Calendario con Demanda_Dia_Ciclo y Valor_Dia_Ciclo_Calendario con Valor_Dia_Ciclo
    df['Demanda_Dia_Calendario'] = df['Demanda_Dia_Ciclo']
    df['Valor_Dia_Ciclo_Calendario'] = df['Valor_Dia_Ciclo'] # Inicializar con el valor original

    # Función auxiliar para aplicar la lógica de cálculo por grupo
    def _calcular_demanda_grupo(group: pd.DataFrame) -> pd.DataFrame:
        group = group.copy()

        dia_inicio_grupo = group['Dia_Inicio'].iloc[0]
        dia_fin_grupo = group['Dia_Fin'].iloc[0]
        demanda_total_ciclo = group['Demanda'].iloc[0]

        # Condición principal: Si fecha_hoy está dentro del rango del ciclo [Dia_Inicio, Dia_Fin]
        if fecha_hoy >= dia_inicio_grupo and fecha_hoy <= dia_fin_grupo:
            # Los días hasta e incluyendo fecha_hoy mantienen Demanda_Dia_Ciclo como Demanda_Dia_Calendario
            mask_dias_hasta_hoy = (group['Dia_Calendario'] <= fecha_hoy)
            group.loc[mask_dias_hasta_hoy, 'Demanda_Dia_Calendario'] = group.loc[mask_dias_hasta_hoy, 'Demanda_Dia_Ciclo']
            group.loc[mask_dias_hasta_hoy, 'Valor_Dia_Ciclo_Calendario'] = group.loc[mask_dias_hasta_hoy, 'Valor_Dia_Ciclo']

            # Suma de la demanda hasta fecha_hoy (inclusive)
            demanda_consumida_hasta_hoy = group.loc[mask_dias_hasta_hoy, 'Demanda_Dia_Calendario'].sum()

            # Caso especial: Si fecha_hoy es el último día del ciclo (fecha_fin)
            if fecha_hoy == dia_fin_grupo:
                # No hay demanda pendiente para distribuir, se habrá "cumplido" el 100% de la demanda hasta hoy.
                # Ya los valores de Demanda_Dia_Calendario y Valor_Dia_Ciclo_Calendario fueron establecidos arriba.
                pass
            else:
                # Calcular la demanda pendiente a distribuir (desde mañana hasta fecha_fin)
                demanda_pendiente_a_distribuir = demanda_total_ciclo - demanda_consumida_hasta_hoy
                
                # Asegurarse de que la demanda pendiente no sea negativa
                demanda_pendiente_a_distribuir = max(0, demanda_pendiente_a_distribuir)

                # Días futuros: desde fecha_hoy + 1 hasta fecha_fin
                fecha_manana = fecha_hoy + timedelta(days=1)
                mask_dias_futuros = (group['Dia_Calendario'] >= fecha_manana) & \
                                    (group['Dia_Calendario'] <= dia_fin_grupo)

                # Suma de los Valor_Dia_Ciclo ORIGINALES para los días futuros
                suma_valor_dia_ciclo_futuro_original = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'].sum()

                if suma_valor_dia_ciclo_futuro_original > 0:
                    # Recalcular Valor_Dia_Ciclo_Calendario para los días futuros
                    # Los nuevos porcentajes deben sumar 1 (100%) para la demanda pendiente
                    recalibrated_valor_dia_ciclo = group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo'] / suma_valor_dia_ciclo_futuro_original
                    group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo_Calendario'] = recalibrated_valor_dia_ciclo

                    # Distribuir la demanda pendiente
                    group.loc[mask_dias_futuros, 'Demanda_Dia_Calendario'] = demanda_pendiente_a_distribuir * recalibrated_valor_dia_ciclo
                else:
                    # Si no hay días futuros con Valor_Dia_Ciclo > 0 para redistribuir,
                    # la demanda pendiente no se distribuye. Estos días quedan en 0 si no tienen valor inicial.
                    group.loc[mask_dias_futuros, 'Demanda_Dia_Calendario'] = 0
                    group.loc[mask_dias_futuros, 'Valor_Dia_Ciclo_Calendario'] = 0
        
        # Casos donde fecha_hoy está fuera del rango [Dia_Inicio, Dia_Fin]
        # (fecha_hoy > dia_fin_grupo o fecha_hoy < dia_inicio_grupo):
        # En estos casos, Demanda_Dia_Calendario y Valor_Dia_Ciclo_Calendario
        # ya fueron inicializados con los valores de Demanda_Dia_Ciclo y Valor_Dia_Ciclo,
        # lo cual es el comportamiento deseado según tus reglas iniciales.

        return group

    # Aplicar la función por grupo
    df_resultado = df.groupby(['Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal'], group_keys=False).apply(_calcular_demanda_grupo)

    # Asegurarse de que el orden de las columnas se mantenga o se añadan al final
    return df_resultado.reset_index(drop=True)

## Procesar todo


In [ ]:

# Obtener demandas
df_demanda_retail = procesar_demanda_retail(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Demanda Retail.xlsx", productos)
df_demanda_vol = procesar_demanda_vol(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Carga de demanda Vol.xlsx", productos)
df_demanda_vd = procesar_demanda_vd(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Carga de demanda VD.xlsx", productos)



In [ ]:
df_demanda_retail

In [ ]:
df_demanda_vol

In [ ]:
df_demanda_vd

In [ ]:
# Obtener demandas explotadas con curva sin calendario
demanda_retail = distribuir_demanda(df_demanda_retail, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Retail.xlsx")
demanda_vol = distribuir_demanda(df_demanda_vol, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin Vol.xlsx")
demanda_vd = distribuir_demanda(df_demanda_vd, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Curva Synplin VD.xlsx")

# Concatenar las 3 demandas
demanda_curva = pd.concat([demanda_retail, demanda_vol, demanda_vd], ignore_index=True)

demanda_curva

In [ ]:
# Cambiar el dataframe con campo días distribuido por producto
demanda_curva_distribuida = transformar_df_dias(demanda_curva)
demanda_curva_distribuida

In [ ]:
# Clasificar días de ciclos según días calendario
demanda_curva_calendario = incorporar_dia_calendario(demanda_curva_distribuida, r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\Calendario Apertura y Cierre 2025.xlsx")
demanda_curva_calendario

In [ ]:
# Calcular demanda de cada día calendario, según la fecha de hoy.
demanda_dia_calendario_recalculada = calcular_demanda_por_dia(demanda_curva_calendario)

demanda_dia_calendario_recalculada.to_excel(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\demanda_explotada_calendarizada.xlsx", index = False)

demanda_dia_calendario_recalculada


In [ ]:
columnas_deseadas = [
    'Año', 'Ciclo', 'Subciclo', 'Código_Kit', 'Código_Producto', 'Canal', 'Demanda',
    'Dia_Ciclo', 'Dia_Calendario', 'Dia_Inicio', 'Dia_Fin',
    'Valor_Dia_Ciclo', 'Valor_Dia_Ciclo_Calendario',
    'Demanda_Dia_Ciclo', 'Demanda_Dia_Calendario'
]

df_filtrado = demanda_dia_calendario_recalculada[columnas_deseadas].to_excel(r"C:\Users\Spider Build\Downloads\Plan Chile\Demandas\demanda_explotada_calendarizada_ordenado.xlsx", index = False)
